In [1]:
# pip install tiktoken -q

In [2]:
# pip install pandas

In [3]:
! python --version

Python 3.11.7


In [4]:
# !pip install langchain-community -q

In [5]:
# pip install langchain -q

In [1]:
import time
import pandas as pd
import json
import os
from langchain_community.chat_models import ChatOllama
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts.chat import (ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate)
from langchain import PromptTemplate
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.schema import (SystemMessage, HumanMessage, AIMessage)
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.docstore.document import Document

In [7]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# def recurs_split(x):
#     text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#         chunk_size=400, 
#         chunk_overlap=50
#     )
#     split_docs = text_splitter.create_documents([x])
#     return(split_docs)

In [2]:
#examples
# df=pd.read_csv("Bio_with_only_reports_csv_1.csv")
df=pd.read_csv("Bio_with_only_reports_csv_1.csv",  encoding_errors= 'replace')
texts=df["V_BIOGRAPHY_repo"].to_list()

In [36]:
# # [Head:Date,  Tail :Crime (specific)  , Relation : CrimeComiitted]
# # [Head: Date, Tail: Decision (like fine , penalty, sentence,bail), Relation: Decision ]
# #[Head:Date, Tail: Crime (specific), Relation : Chargedwithcrime ]  



# examples = [
#      {"input":"aug 2013 - declared as proclaimed offender by karachi court for alleged involvement in theft, \
#                           arms and ammunition. jun 2022 - no further information reported. ",
#       "output":[{"head":"August 2013","head_type":"Date","relation":"Decision","tail":"proclaimed offender","tial_type":"Decision"},{"head":"proclaimed offender","head_type":"Decision",\
#                                                         "relation":"CrimeCommitted","tail":"Theft,arms and ammunition","tail_type":"Crime"}]

#           },
#     {
#      "input":' 2014 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping.',
#       "output":[{"head":"2014","head_type":"Date","relation":"Decision","tail":"proclaimed offender","tial_type":"Decision"},
#        {"head":"proclaimed offender","head_type":"Decision","relation":"CrimeCommitted","tail":"kidnapping","tail_type":"Crime"}]},

#     {
#         "input": 'Oct 2022 - sentenced by Sampit District Court to 5 years \
#             imprisonment and fined IDR1b in lieu of additional 3 months imprisonment for methamphetamine trafficking (Feb - 2021). \
#             and last year he got an award as the Best Talent'
#         ,
#         "output":[{"head": "October 2022",
#         "head_type": "Date",
#         "relation": "Decision",
#         "tail": "sentenced,imprisonment,fined",
#         "tail_type": "Decision"},
#         {"head":"February-2021",
#          "head_type":"Date",
#          "relation":"CrimeCommited",
#          "tail":"methamphetamine trafficking",
#          "tail_type":"Crime"}
#     ]},    {"input":" jan 2009 - sentenced by cbi to 5 years rigorous imprisonment and fined rs 400,000 for fraud,\
#              criminal conspiracy and forgery in obtaining illegally a duty drawback of rs 1.3m.",
#      "output":[{"head":"January 2009","head_type":"Date","relation":"Decision","Tail":"imprisionment and fined","Tial_type":"Decision"},{"head":"imprisionment and fined","head_type":"Decision",\
#                                                                                                                 "relation":"CrimeCommited","tail":"Criminal conspiracy and forgery","tail_type":"Crime" }]\
    
#     },
#     {
#         "input": 'dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. \
#         case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - \
#         convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and \
#         fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.'\
#              , "output":[
#         {"head": "July 2019-October 2019",
#         "head_type": "Date",
#         "relation": "CrimeCommitted",
#         "tail": "Bribery",
#         "tail_type": "Crime"},{ 
#             "head":"Decemeber 2019",
#              "head_type":"Date",
#              "relation" :"ChargedwithCrime",
#              "tail":"Bribery",
#              "tail_type":"Crime"}
            
#     ,{       "head":"March 2022",
#              "head_type":"Date",
#              "relation" :"Decision",
#              "tail":"imprisionment,fined",
#              "tail_type":"Decision"
        
#     }]},{"input":'aug 2021 - first information report filed by central bureau of investigation for wrongful restraint, grievous\
#      hurt by dangerous weapon, attempt to murder and criminal intimidation. jun 2022 - no further information reported.',
#          "output":[{
#         "head":"August 2021",
#              "head_type":"Date",
#              "relation" :"ChargedwithCrime",
#              "tail":"attempt to murder and criminal intimidation",
#              "tail_type":"Crime"}]},
#     {"input":'dec 2011 - held responsible for misrepresentation in business records. warned and fined cny20,000 by cbirc hubei bureau. jun 2022 - no further information reported.'
# ,"output":[{"head":"December 2011","head_type":"Date","relation":"ChargedwithCrime","tail":"misrepresentation in business records","tail_type":"CrimeCommited"},
#           {"head":"misrepresentation in business records","head_type":"CrimeCommited","relation":"Decision","tail":"warned and fined","tail_type":"Decision"}]},
    
#     {"input":" year 2009 - reportedly declared as proclaimed offender by ghotki court for alleged involvement in dacoity. jun 2022 - no further information reported.",
#      "output":[{"head":"Year 2009","head_type":"Date","relation":"Decision","tail":"proclaimed offender","tail_type":"Decision"},{
#          "head":"proclaimed offender","head_type":"Decision","relation":"CrimeCommited","tail":"Dacoity","tail_type":"Crime"
#      }]},
   
        
#     ]




## Prompt-1 

In [37]:
# [Head:Date,  Tail :Crime (specific)  , Relation : CrimeComiitted]
# [Head: Date, Tail: Decision (like fine , penalty, sentence,bail), Relation: Decision ]
#[Head:Date, Tail: Crime (specific), Relation : Chargedwithcrime ]  



examples = [ 
     {"input":' 2007 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. aug 2022 - no further information reported.',
      "output":[{"head":"2007","head_type":"Date","relation":"Convicted","tail":"kidnapping","tial_type":"Crime"}]

          },
    {
     "input":' 2014 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping.',
      "output":[{"head":"2014","head_type":"Date","relation":"Convicted","tail":"kidnapping","tial_type":"Crime"}]
    
     }]       





In [40]:
#new entities list
# entity_types = ['primary name', 'age', 'nationality', 'location', 'sentence', 'crime', 'date', 'amount (value)', 'date of arrest', 'alias', 
#                 'designation', 'Connected Party (Organization)', 'Connected Party (Individual)']
# entity_types = ['parties involved in transaction', 'Residence location', 'primary name', 'activity location', 'time reported of location', 
#                 'time period of residence', 'is deceased', 'employer name', 'alias', 'Address location', 'deceased date', 'id type', 'martial status', 
#                 'age', 'activity at location', 'time period of address', 'gender', 'transaction amount', 'issued on', 'age of', 'employer type', 
#                 'employment reported on', 'location of', 'marriage date', 'id belongs to', 'relative or closely associated person', 'name of deceased', 
#                 'identification no', 'activity', 'null', 'reported time period', 'organization (terrorist)', 'residence of', 'age reported on date', 
#                 'designation', 'date', 'expiry on', 'employement period', 'is PEP', 'deceased status', 'none', 'location', 'employer of', 'address of', 
#                 'deceased status reported on date', 'marital status reported on date', 'employment period', 'biography_person', 
#                 'organization (government)', 'organization (others)', 'son of']

# relation_types = ['none', 'age of', 'nationality of', 'location of', 'sentence of', 'crime of', 'date of announcing', 'amount (value) of', 
#                   'date of arrest of', 'alias of', 'designation of', 'designation in', 'connection with']
# relation_types  = ['none', 'citizenship', 'brother of', 'place of birth of', 'from', 'related or closely associated to', 'joint account holder with', 
#                    'gender of', 'spouse of', 'alias of', 'son of', 'nationality', 'native of', 'date of birth of', 'daughter of', 'father of']

In [5]:
# print(len(entity_types), len(relation_types))

In [43]:
#this piece works

# system_prompt = PromptTemplate(
#     template = """
#     You are a top-tier algorithm designed for extracting information in structured formats.
#     Your task is to identify the entities and relations requested with the user prompt, from a given text.
#     You must generate the output in a JSON containing a list with JSON objects having the following keys: "head", "head_type", "relation", "tail", and "tail_type", as is also specified in the examples.
#     The "head" key must contain the text of the extracted entity with one of the types from the provided list in the user prompt. 
#     The "head_type" key must contain the type of the extracted head entity which must be one of the types from {entity_types}.
#     The "relation" key must contain the type of relation between the "head" and the "tail" which must be one of the relations from {relation_types}.
#     The "tail" key must represent the text of an extracted entity which is the tail of the relation, and the "tail_type" key must contain the type of the tail entity from {entity_types}. 
    
#     Attempt to extract as many entities and relations as you can. Only use entity and relation type as mentioned. 
#     DO NOT RETURN ANY OTHER ENTITY OR RELATION TYPE.
    
#     IMPORTANT NOTES:
#     - Don't add any explanation and text. 
#     - Strictly adhere to formats in examples.
#     - If information about any field is not present in the text, return 'None' for those fields. 
#     - If information about any field is not present in the text, DO NOT publish any information from the added examples. 
#     """,
#     input_variables=["entity_types", "relation_types"],
# )

# system_message_prompt = SystemMessagePromptTemplate(prompt = system_prompt)

# human_prompt = PromptTemplate(
#     template = """ Based on the following example, extract entities and relations from the provided text.\n\n

#     Use only the following entity types, don't return any other entity type:
#     # ENTITY TYPES:
#     {entity_types}

#     Use only the following relation types, don't return any other relation type:
#     # RELATION TYPES:
#     {relation_types}

#     Below are a number of examples of text and their extracted entities and relationshhips.
#     {examples}

#     For the following text, generate extract entitites and relations as in the provided example.\nText: {text}""",
#     input_variables=["entity_types", "relation_types", "examples", "text"],
# )

# human_message_prompt = HumanMessagePromptTemplate(prompt=human_prompt)

# chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# model = ChatOllama(model = "llama3", temperature=0)
# chain = LLMChain(llm=model, prompt=chat_prompt)

# prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# llm = ChatOllama(model = "llama3", temperature=0)
# chain = prompt | llm

In [38]:
system_prompt ='''You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
Your task is to identify the entities and relations from the input text. You must generate the output as a list containing
JSON objects. Each JSON object should be in format {"head":entity,"head_type":type of entity assigned to head, "tail":entity, "tail_type":type of entity assigned to tail,"relation":relation between entities asigned to head and tail}.
'''

In [39]:
class ExtractedInfo(BaseModel):
    head: str = Field(description="extracted first or head entity like Decmeber-2022 etc.")
    head_type: str = Field(description="type of the extracted head entity like Date etc.")
    relation: str = Field(description="relation between the head and the tail entities or if tail is none then relation be also none.")
    tail: str = Field(description="extracted second or tail entity like Bribery,Fined,Imprisioned, None etc.")
    tail_type: str = Field(description="type of the extracted tail entity like Crime, Sentence, None etc.")
    
parser = JsonOutputParser(pydantic_object=ExtractedInfo)

In [40]:
# all_mrgd_ot_gpu
llm=ChatOllama(model="llama3",temperature=0,format="json")
human_prompt=PromptTemplate(template="""
Below are a list of examples,each having a JSON object with input and output as keys.Input key takes {input}
and output key generates is list of JSON having extracted entity and relations.Generate output as per this format.

examples: {examples}
Use the following relation types, don't use other relation that is not defined below:
# RELATION TYPES:
{rel_types}

For the following input, extract entities and relationships as described in examples.
{format_instructions}
Input text: {input}
Identify only the relations mentioned in {rel_types} .
"""
,input_variables=["input"],partial_variables={"examples":examples,"rel_types":["Convicted","CrimeCommited","Decision","ChargedWithCrime"],
        "format_instructions": parser.get_format_instructions()})


human_message_prompt=HumanMessagePromptTemplate(prompt=human_prompt)


# "Date" and "Crime"  with relation as "Convicted".Don't provide any other entity and relationship

In [41]:
system_message=SystemMessage(content=system_prompt)
chat_prompt=ChatPromptTemplate.from_messages([system_message,human_message_prompt])

In [42]:


# prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
# chain = chat_prompt | llm

In [14]:
print(chain.run("""
 2009 - reportedly declared as proclaimed offender by ghotki court for alleged involvement in dacoity. jun 2022 - no further information reported.
"""))

D:\anaconda\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Here is the output:

```
[
    {
        "input": "2009 - reportedly declared as proclaimed offender by ghotki court for alleged involvement in dacoity. jun 20,22 - no further information reported.",
        "output": [
            {
                "head": "2009",
                "head_type": "Date",
                "relation": "Convicted",
                "tail": "dacoity",
                "tail_type": "Crime"
            }
        ]
    }
]
```

Note that I've extracted the year "2009" as the head entity, with type "Date", and the crime "dacoity" as the tail entity, with type "Crime". The relation between them is "Convicted".


In [15]:
 print(chain.run("""
jul 2022 - held responsible for company's violation of misrepresentation in financial records. warned and fined cny80,000 by cbirc lianyungang bureau.
"""))

Here is the output:

```
[
    {
        "input": "jul 20,22 - held responsible for company's violation of misrepresentation in financial records. warned and fined cny80,000 by cbirc lianyungang bureau.",
        "output": [
            {
                "head": "Jul-20,22",
                "head_type": "Date",
                "relation": "Convicted",
                "tail": "misrepresentation in financial records",
                "tail_type": "Crime"
            }
        ]
    }
]
```

Note that I've extracted the date as "Jul-20,22" and the crime as "misrepresentation in financial records", and assigned the relation as "Convicted".


In [16]:
print(
    chain.run("""  2014 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. jun 2022 - no further information reported.    """)
)

Here is the output:

```
[
    {
        "input": "2014 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. jun 20,22 - no further information reported.",
        "output": [
            {
                "head": "2014",
                "head_type": "Date",
                "relation": "Convicted",
                "tail": "kidnapping",
                "tail_type": "Crime"
            }
        ]
    }
]
```

Note that I've only extracted one entity and relationship, as per the input text. The relation is "Convicted" with the head being the date "2014", the tail being the crime "kidnapping", and both are of type "Date" and "Crime" respectively.


In [17]:
print(
    chain.run("""  2010 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. jul 2022 - no further information reported.
    """))

Here is the output:

```
[
    {
        "input": "   2010 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. jul 20,22 - no further information reported.",
        "output": [
            {
                "head": "2010",
                "head_type": "Date",
                "relation": "Convicted",
                "tail": "kidnapping",
                "tail_type": "Crime"
            }
        ]
    }
]
```

Note that I've only extracted one entity and relationship, as there is only one relevant piece of information in the input text.


In [18]:
print(chain.run("feb 2013 - sentenced to 4 years imprisonment by central district court of barnaul for illegal sale of narcotics committed by organised group. jul 2022 - no further information reported."))

Here is the output in the required format:

```
[
    {
        "input": "feb 2013 - sentenced to 4 years imprisonment by central district court of barnaul for illegal sale of narcotics committed by organised group. jul 20,22 - no further information reported.",
        "output": [
            {
                "head": "Feb 2013",
                "head_type": "Date",
                "relation": "Convicted",
                "tail": "Illegal sale of narcotics",
                "tail_type": "Crime"
            }
        ]
    }
]
```

Note that I've only extracted one entity and relationship, as there is only one relevant piece of information in the input text.


In [19]:
print(
chain.run(""" mar 2022 - sentenced by an giang provincial people's court to 2 years imprisonment for organising illegal gambling (jun 2020 - aug 2020). fined vnd20m.""")
)

Here is the output in the required format:

```
[
    {
        "input": "mar 20,22 - sentenced by an giang provincial people's court to 2 years imprisonment for organising illegal gambling (jun 20,20 - aug 20,20). fined vnd20m.",
        "output": [
            {
                "head": "Mar-20,22",
                "head_type": "Date",
                "relation": "Convicted",
                "tail": "organising illegal gambling",
                "tail_type": "Crime"
            }
        ]
    }
]
```

Note: I assumed the date format to be "Month-Year" and extracted the relevant entities and relationships as per the given schema.


In [20]:
print(
chain.run("""jun 2020 - first information report filed by central bureau of investigation for criminal conspiracy, cheating, destruction of evidence. aug 2022 - no further information reported."""))

Here is the output in the required format:

```
[
    {
        "input": "jun 20,20 - first information report filed by central bureau of investigation for criminal conspiracy, cheating, destruction of evidence. aug 20,22 - no further information reported.",
        "output": [
            {"head": "Jun 20,20", "head_type": "Date", "relation": "Convicted", "tail": "Criminal Conspiracy", "tail_type": "Crime"},
            {"head": "Aug 20,22", "head_type": "Date", "relation": "Convicted", "tail": "Destruction of Evidence", "tail_type": "Crime"}
        ]
    }
]
```

Note that I've assumed the entities are "Jun 20,20" and "Aug 20,22" as dates, and "Criminal Conspiracy" and "Destruction of Evidence" as crimes.


In [21]:
print(
chain.run(""" 2013 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in banditry. aug 2022 - no further information reported."""))

Here is the output:

```
[
    {
        "input": "2013 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in banditry. aug 20,22 - no further information reported.",
        "output": [
            {
                "head": "2013",
                "head_type": "Date",
                "relation": "Convicted",
                "tail": "banditry",
                "tail_type": "Crime"
            }
        ]
    }
]
```

Note that I've extracted the year "2013" as the head entity, with type "Date", and the crime "banditry" as the tail entity, with type "Crime". The relation between them is "Convicted".


In [22]:
print(
chain.run("""  oct 2012 - declared as proclaimed offender by karachi court for alleged involvement in cheating. aug 2022 - no further information reported."""))

Here is the output:

```
[
    {
        "head": "oct 2012",
        "head_type": "Date",
        "relation": "Convicted",
        "tail": "cheating",
        "tail_type": "Crime"
    },
    {
        "head": "aug 20,22",
        "head_type": "Date",
        "relation": "Convicted",
        "tail": "None",
        "tail_type": "None"
    }
]
```

Note that I've used the relation type "Convicted" for both entities, and the tail entity is either "cheating" or "None".


In [43]:
texts=df.loc[:100,"V_BIOGRAPHY_repo"].to_list()

In [45]:
llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
l1=[]
bio1=[]
ind1=[]
index1=0
for i in texts:
    print(index)
    
    try:
       output=chain.run(i)
       l1.append(output)
       bio1.append(i)
       ind1.append(index1)
       index1=index1+1
    except:
        index1=index1+1
        

101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101
101


In [46]:
df_res1=pd.DataFrame({})
df_res1["Biography"]=bio1
df_res1["groups"]=l1
df_res1["index"]=ind1

In [47]:
df_res1.to_csv("only_crime.csv")

## Prompt -2

In [3]:

examples_2 = [ 
     {"input":' 2007 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. aug 2022 - no further information reported.',
      "output":[{"head":"2007","head_type":"Date","relation":"Decision","tail":"proclaimed offender","tial_type":"Decision"},{"head":"2007","head_type":"Date","relation":"Convicted","tail":"kidnapping","tial_type":"Crime"}]

          },
    {
     "input":' 2014 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping.',
      "output":[{"head":"2014","head_type":"Date","relation":"Decision","tail":"proclaimed offender","tial_type":"Decision"},{"head":"2014","head_type":"Date","relation":"Convicted","tail":"kidnapping","tial_type":"Crime"}]
    
     }    ,



    {"input":"aug 2013 - declared as proclaimed offender by karachi court for alleged involvement in theft, \
                          arms and ammunition. jun 2022 - no further information reported. ",
      "output":[{"head":"August 2013","head_type":"Date","relation":"Decision","tail":"proclaimed offender","tial_type":"Decision"},{"head":"August 2013","head_type":"Date",\
                                                        "relation":"Convicted","tail":"Theft,arms and ammunition","tail_type":"Crime"}]

          },
    {
     "input":' 2014 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping.',
      "output":[{"head":"2014","head_type":"Date","relation":"Decision","tail":"proclaimed offender","tial_type":"Decision"},
       {"head":"2014","head_type":"Date","relation":"Convicted","tail":"kidnapping","tail_type":"Crime"}]},

    {
        "input": 'Oct 2022 - sentenced by Sampit District Court to 5 years \
            imprisonment and fined IDR1b in lieu of additional 3 months imprisonment for methamphetamine trafficking (Feb - 2021). \
            and last year he got an award as the Best Talent'
        ,
        "output":[{"head": "October 2022",
        "head_type": "Date",
        "relation": "Decision",
        "tail": "sentenced,imprisonment,fined",
        "tail_type": "Decision"},
        {"head":"February-2021",
         "head_type":"Date",
         "relation":"CrimeCommited",
         "tail":"methamphetamine trafficking",
         "tail_type":"Crime"}
    ]},    {"input":" jan 2009 - sentenced by cbi to 5 years rigorous imprisonment and fined rs 400,000 for fraud,\
             criminal conspiracy and forgery in obtaining illegally a duty drawback of rs 1.3m.",
     "output":[{"head":"January 2009","head_type":"Date","relation":"Decision","Tail":"imprisionment and fined","Tial_type":"Decision"},{"head":"Jan 2009","head_type":"Date",\
                                                                                                                "relation":"Convicted","tail":"obtaining illegally a duty drawback","tail_type":"Crime" }]\
    
    },
    {
        "input": 'dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. \
        case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - \
        convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and \
        fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.'\
             , "output":[
        {"head": "July 2019-October 2019",
        "head_type": "Date",
        "relation": "CrimeCommitted",
        "tail": "Bribery",
        "tail_type": "Crime"},{ 
            "head":"Decemeber 2019",
             "head_type":"Date",
             "relation" :"ChargedwithCrime",
             "tail":"Bribery",
             "tail_type":"Crime"}
            
    ,{       "head":"March 2022",
             "head_type":"Date",
             "relation" :"Decision",
             "tail":"imprisionment,fined",
             "tail_type":"Decision"
        
    }]},{"input":'aug 2021 - first information report filed by central bureau of investigation for wrongful restraint, grievous\
     hurt by dangerous weapon, attempt to murder and criminal intimidation. jun 2022 - no further information reported.',
         "output":[{
        "head":"August 2021",
             "head_type":"Date",
             "relation" :"ChargedwithCrime",
             "tail":"attempt to murder and criminal intimidation",
             "tail_type":"Crime"}]},
    {"input":'dec 2011 - held responsible for misrepresentation in business records. warned and fined cny20,000 by cbirc hubei bureau. jun 2022 - no further information reported.'
,"output":[{"head":"December 2011","head_type":"Date","relation":"Convicted","tail":"misrepresentation in business records","tail_type":"Crime"},
          {"head":"December 2011","head_type":"Date","relation":"Decision","tail":"warned and fined","tail_type":"Decision"}]},
    
    {"input":" year 2009 - reportedly declared as proclaimed offender by ghotki court for alleged involvement in dacoity. jun 2022 - no further information reported.",
     "output":[{"head":"Year 2009","head_type":"Date","relation":"Decision","tail":"proclaimed offender","tail_type":"Decision"},{
         "head":"Year 2009","head_type":"Date","relation":"Convicted","tail":"Dacoity","tail_type":"Crime"
     }]},
   
        
    ]



In [4]:

# "Date" and "Crime"  with relation as "Convicted".Don't provide any other entity and relationship

In [7]:
system_prompt ='''You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
Your task is to identify the entities and relations from the input text. You must generate the output as a list containing
JSON objects. Each JSON object should be in format {"head":entity,"head_type":type of entity assigned to head, "tail":entity, "tail_type":type of entity assigned to tail,"relation":relation between entities asigned to head and tail}.
'''

In [8]:
class ExtractedInfo(BaseModel):
    head: str = Field(description="extracted first or head entity like Decmeber-2022 etc.")
    head_type: str = Field(description="type of the extracted head entity like Date etc.")
    relation: str = Field(description="relation between the head and the tail entities or if tail is none then relation be also none.")
    tail: str = Field(description="extracted second or tail entity like Bribery,Fined,Imprisioned, None etc.")
    tail_type: str = Field(description="type of the extracted tail entity like Crime, Sentence, None etc.")
    
parser = JsonOutputParser(pydantic_object=ExtractedInfo)

In [9]:
# all_mrgd_ot_gpu
llm=ChatOllama(model="llama3",temperature=0,format="json")
human_prompt=PromptTemplate(template="""
Below are a list of examples,each having a JSON object with input and output as keys.Input key takes {input}
and output key generates is list of JSON having extracted entity and relations.Generate output as per this format.

examples: {examples}
Use the following relation types, don't use other relation that is not defined below:
# RELATION TYPES:
{rel_types}

For the following input, extract entities and relationships as described in examples.
{format_instructions}
Input text: {input}
Identify only the relations mentioned in {rel_types} .
"""
,input_variables=["input"],partial_variables={"examples":examples_2,"rel_types":["Convicted","CrimeCommited","Decision","ChargedWithCrime"],
        "format_instructions": parser.get_format_instructions()})


human_message_prompt=HumanMessagePromptTemplate(prompt=human_prompt)


# "Date" and "Crime"  with relation as "Convicted".Don't provide any other entity and relationship

In [10]:
system_message=SystemMessage(content=system_prompt)
chat_prompt=ChatPromptTemplate.from_messages([system_message,human_message_prompt])

In [11]:


# prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
# chain = chat_prompt | llm

C:\Users\admin\anaconda3\envs\TM_NS\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [12]:
%%time
print(chain.run("""
 2009 - reportedly declared as proclaimed offender by ghotki court for alleged involvement in dacoity. jun 2022 - no further information reported.
"""))

C:\Users\admin\anaconda3\envs\TM_NS\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{
    "output": [
        {
            "head": "2009",
            "head_type": "Date",
            "relation": "Decision",
            "tail": "proclaimed offender",
            "tail_type": "Decision"
        },
        {
            "head": "2009",
            "head_type": "Date",
            "relation": "Convicted",
            "tail": "dacoity",
            "tail_type": "Crime"
        }
    ]
}


In [17]:
%%time
print(chain.run("""
jul 2022 - held responsible for company's violation of misrepresentation in financial records. warned and fined cny80,000 by cbirc lianyungang bureau.
"""))

{
    "output": [
        {
            "head": "jul 2022",
            "head_type": "Date",
            "relation": "Decision",
            "tail": "held responsible for company's violation of misrepresentation in financial records.",
            "tail_type": "Crime"
        },
        {
            "head": "jul 2022",
            "head_type": "Date",
            "relation": "Convicted",
            "tail": "violation of misrepresentation in financial records",
            "tail_type": "Crime"
        }
    ]
}
CPU times: total: 266 ms
Wall time: 8.49 s


In [18]:
%%time
print(
    chain.run("""  2014 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. jun 2022 - no further information reported.    """)
)

{
    "input": "2014 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. jun 2022 - no further information reported.",
    "output": [
        {
            "head": "2014",
            "head_type": "Date",
            "relation": "Decision",
            "tail": "proclaimed offender",
            "tail_type": "Decision"
        },
        {
            "head": "2014",
            "head_type": "Date",
            "relation": "Convicted",
            "tail": "kidnapping",
            "tail_type": "Crime"
        }
    ]
}
CPU times: total: 281 ms
Wall time: 8.89 s


In [15]:
print(
    chain.run("""  2010 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. jul 2022 - no further information reported.
    """))

{
    "input": "2010 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. jul 2022 - no further information reported.",
    "output": [
        {
            "head": "2010",
            "head_type": "Date",
            "relation": "Decision",
            "tail": "proclaimed offender",
            "tail_type": "Decision"
        },
        {
            "head": "2010",
            "head_type": "Date",
            "relation": "Convicted",
            "tail": "kidnapping",
            "tail_type": "Crime"
        }
    ]
}


In [16]:
print(chain.run("feb 2013 - sentenced to 4 years imprisonment by central district court of barnaul for illegal sale of narcotics committed by organised group. jul 2022 - no further information reported."))

{
    "input": "feb 2013 - sentenced to 4 years imprisonment by central district court of barnaul for illegal sale of narcotics committed by organised group. jul 2022 - no further information reported.",
    "output": [
        {
            "head": "Feb 2013",
            "head_type": "Date",
            "relation": "Decision",
            "tail": "sentenced,imprisonment",
            "tail_type": "Decision"
        },
        {
            "head": "Feb 2013",
            "head_type": "Date",
            "relation": "CrimeCommited",
            "tail": "illegal sale of narcotics",
            "tail_type": "Crime"
        }
    ]
}


In [19]:
print(
chain.run(""" mar 2022 - sentenced by an giang provincial people's court to 2 years imprisonment for organising illegal gambling (jun 2020 - aug 2020). fined vnd20m.""")
)

{
    "input": [
        {
            "head": "Mar 2022",
            "head_type": "Date",
            "relation": "Decision",
            "tail": "sentenced,imprisonment,fined",
            "tail_type": "Decision"
        },
        {
            "head": "Jun 2020 - Aug 2020",
            "head_type": "Date",
            "relation": "CrimeCommited",
            "tail": "organising illegal gambling",
            "tail_type": "Crime"
        }
    ]
}


In [20]:
print(
chain.run("""jun 2020 - first information report filed by central bureau of investigation for criminal conspiracy, cheating, destruction of evidence. aug 2022 - no further information reported."""))

{
    "output": [
        {
            "head": "Jun 2020",
            "head_type": "Date",
            "relation": "Decision",
            "tail": "first information report filed by central bureau of investigation for criminal conspiracy, cheating, destruction of evidence",
            "tail_type": "Crime"
        }
    ]
}


In [21]:
print(
chain.run(""" 2013 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in banditry. aug 2022 - no further information reported."""))

{
    "output": [
        {
            "head": "2013",
            "head_type": "Date",
            "relation": "Decision",
            "tail": "proclaimed offender",
            "tail_type": "Decision"
        },
        {
            "head": "2013",
            "head_type": "Date",
            "relation": "Convicted",
            "tail": "banditry",
            "tail_type": "Crime"
        }
    ]
}


In [22]:
print(
chain.run("""  oct 2012 - declared as proclaimed offender by karachi court for alleged involvement in cheating. aug 2022 - no further information reported."""))

{
    "output": [
        {
            "head": "Oct 2012",
            "head_type": "Date",
            "relation": "Decision",
            "tail": "proclaimed offender",
            "tail_type": "Decision"
        },
        {
            "head": "Oct 2012",
            "head_type": "Date",
            "relation": "Convicted",
            "tail": "cheating",
            "tail_type": "Crime"
        }
    ]
}


In [23]:
texts=df.loc[:100,"V_BIOGRAPHY_repo"].to_list()

In [24]:
llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
l=[]
bio=[]
ind=[]
index=0
for i in texts[]:
    print(index)
    
    try:
       output=chain.run(i)
       l.append(output)
       bio.append(i)
       ind.append(index)
       index=index+1
    except:
        index=index+1
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [28]:
for i in texts[51:]:
    print(index)
    
    try:
       output=chain.run(i)
       l.append(output)
       bio.append(i)
       ind.append(index)
       index=index+1
    except:
        index=index+1

51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [31]:
df_res=pd.DataFrame({})
df_res["Biography"]=bio
df_res["groups"]=l
df_res["index"]=ind

In [33]:
df_res.to_csv("final_res_4.csv")

## Prompt-3

In [2]:
import random
random.seed(7)
examples_3 = [ 
     {"input":' 2007 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. aug 2022 - no further information reported.',
      "output":[{"head":"2007","head_type":"Date","relation":"Judgement","tail":"proclaimed offender","tial_type":"Decision"},{"head":"2007","head_type":"Date","relation":"Convicted","tail":"kidnapping","tial_type":"Crime"}]

          },
    



    {"input":"aug 2013 - declared as proclaimed offender by karachi court for alleged involvement in theft, \
                          arms and ammunition. jun 2022 - no further information reported. ",
      "output":[{"head":"August 2013","head_type":"Date","relation":"Judgement","tail":"proclaimed offender","tial_type":"Decision"},{"head":"August 2013","head_type":"Date",\
                                                        "relation":"Convicted","tail":"Theft,arms and ammunition","tail_type":"Crime"}]

          },
     {
     "input":' may 2016 - fined rub30,000 by murom city court for organisation of illegal migration (nov 2015 - dec 2015).',
      "output":[{"head":"May 2016","head_type":"Date","relation":"Judgement","tail":"Fined","tial_type":"Decision"},{"head":"November 2015 - December 2015","head_type":"Date","relation":"CrimeCommitted","tail":"illegal migration","tial_type":"Crime"}]
    
     } ,
    {
     "input":' 2014 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping.',
      "output":[{"head":"2014","head_type":"Date","relation":"Judgement","tail":"proclaimed offender","tial_type":"Decision"},
       {"head":"2014","head_type":"Date","relation":"Convicted","tail":"kidnapping","tail_type":"Crime"}]},

    {
        "input": 'Oct 2022 - sentenced by Sampit District Court to 5 years \
            imprisonment and fined IDR1b in lieu of additional 3 months imprisonment for methamphetamine trafficking (Feb - 2021). \
            and last year he got an award as the Best Talent'
        ,
        "output":[{"head": "October 2022",
        "head_type": "Date",
        "relation": "Judgement",
        "tail": "sentenced,imprisonment,fined",
        "tail_type": "Decision"},
        {"head":"February-2021",
         "head_type":"Date",
         "relation":"CrimeCommited",
         "tail":"methamphetamine trafficking",
         "tail_type":"Crime"}
    ]},    {"input":" jan 2009 - sentenced by cbi to 5 years rigorous imprisonment and fined rs 400,000 for fraud,\
             criminal conspiracy and forgery in obtaining illegally a duty drawback of rs 1.3m.",
     "output":[{"head":"January 2009","head_type":"Date","relation":"Judgement","Tail":"imprisionment and fined","Tial_type":"Decision"},{"head":"Jan 2009","head_type":"Date",\
                                                                                                                "relation":"Convicted","tail":"obtaining illegally a duty drawback","tail_type":"Crime" }]\
    
    },
    {
        "input": 'dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. \
        case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - \
        convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and \
        fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.'\
             , "output":[
        {"head": "July 2019-October 2019",
        "head_type": "Date",
        "relation": "CrimeCommitted",
        "tail": "Bribery",
        "tail_type": "Crime"},{ 
            "head":"Decemeber 2019",
             "head_type":"Date",
             "relation" :"ChargedwithCrime",
             "tail":"Bribery",
             "tail_type":"Crime"}
            
    ,{       "head":"March 2022",
             "head_type":"Date",
             "relation" :"Judgement",
             "tail":"imprisionment,fined",
             "tail_type":"Decision"
        
    }]},{"input":'aug 2021 - first information report filed by central bureau of investigation for wrongful restraint, grievous\
     hurt by dangerous weapon, attempt to murder and criminal intimidation. jun 2022 - no further information reported.',
         "output":[{
        "head":"August 2021",
             "head_type":"Date",
             "relation" :"ChargedwithCrime",
             "tail":"attempt to murder and criminal intimidation",
             "tail_type":"Crime"}]},
    {"input":'dec 2011 - held responsible for misrepresentation in business records. warned and fined cny20,000 by cbirc hubei bureau. jun 2022 - no further information reported.'
,"output":[{"head":"December 2011","head_type":"Date","relation":"Convicted","tail":"misrepresentation in business records","tail_type":"Crime"},
          {"head":"December 2011","head_type":"Date","relation":"Judgement","tail":"warned and fined","tail_type":"Decision"}]},
    
    {"input":" year 2009 - reportedly declared as proclaimed offender by ghotki court for alleged involvement in dacoity. jun 2022 - no further information reported.",
     "output":[{"head":"Year 2009","head_type":"Date","relation":"Judgement","tail":"proclaimed offender","tail_type":"Decision"},{
         "head":"Year 2009","head_type":"Date","relation":"Convicted","tail":"Dacoity","tail_type":"Crime"
     }]},
   
        
    ]



In [3]:
system_prompt ='''You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
Your task is to identify the entities and relations from the input text. You must generate the output as a list containing
JSON objects. Each JSON object should be in format {"head":entity,"head_type":type of entity assigned to head, "tail":entity, "tail_type":type of entity assigned to tail,"relation":relation between entities asigned to head and tail}.
'''

In [4]:
class ExtractedInfo(BaseModel):
    head: str = Field(description="extracted first or head entity like Decmeber-2022 etc.")
    head_type: str = Field(description="type of the extracted head entity like Date etc.")
    relation: str = Field(description="relation between the head and the tail entities or if tail is none then relation be also none.")
    tail: str = Field(description="extracted second or tail entity like Bribery,Fined,Imprisioned, None etc.")
    tail_type: str = Field(description="type of the extracted tail entity like Crime, Sentence, None etc.")
    
parser = JsonOutputParser(pydantic_object=ExtractedInfo)

In [5]:
# all_mrgd_ot_gpu
llm=ChatOllama(model="llama3",temperature=0,format="json")
human_prompt=PromptTemplate(template="""
Below are a list of examples,each having a JSON object with input and output as keys.Input key takes {input}
and output key generates is list of JSON having extracted entity and relations.Generate output as per this format.

examples: {examples}
Use the following relation types, don't use other relation that is not defined below:
# RELATION TYPES:
{rel_types}

For the following input, extract entities and relationships as described in examples.
{format_instructions}
Input text: {input}
Identify only the relations mentioned in {rel_types} .
"""
,input_variables=["input"],partial_variables={"examples":examples_3,"rel_types":["Convicted","CrimeCommited","Judgement","ChargedWithCrime"],
        "format_instructions": parser.get_format_instructions()})


human_message_prompt=HumanMessagePromptTemplate(prompt=human_prompt)


# "Date" and "Crime"  with relation as "Convicted".Don't provide any other entity and relationship

In [6]:
system_message=SystemMessage(content=system_prompt)
chat_prompt=ChatPromptTemplate.from_messages([system_message,human_message_prompt])

In [7]:


# prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
# chain = chat_prompt | llm

C:\Users\admin\anaconda3\envs\TM_NS\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [8]:
print(
chain.run(df["V_BIOGRAPHY_repo"].loc[8]))
#ok

NameError: name 'df' is not defined

In [117]:
print(
chain.run(df["V_BIOGRAPHY_repo"].loc[19]))
# missing entity fro 2019-2020

{
    "output": [
        {
            "head": "Jun 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board (2019 - 2020)",
            "tail_type": "Crime"
        },
        {
            "head": "Jun 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "pleaded guilty",
            "tail_type": "Decision"
        }
    ]
}


In [93]:
df["V_BIOGRAPHY_repo"].loc[19]

' jun 2022 - pleaded guilty at the magistrate court to the criminal charges of instigating employees in practicing dentistry without authorisation from the brunei medical board (2019 - 2020). fined bnd24,000, in default, 24 months imprisonment.'

In [63]:
print(
chain.run(df["V_BIOGRAPHY_repo"].loc[24]))
# ok

{
    "output": [
        {
            "head": "Jun 2020",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "Criminal Conspiracy, Cheating, Destruction of Evidence",
            "tail_type": "Crime"
        }
    ]
}


In [64]:
print(
chain.run(df["V_BIOGRAPHY_repo"].loc[29]))
# ok

{
    "output": [
        {
            "head": "Aug 2022",
            "head_type": "Date",
            "relation": "ChargeSheetFiled",
            "tail": "abuse of authority (ciaa) for embezzled of fund",
            "tail_type": "Crime"
        }
    ]
}


In [74]:
df["V_BIOGRAPHY_repo"].loc[30]

' may 2016 - fined rub30,000 by murom city court for organisation of illegal migration (nov 2015 - dec 2015).'

In [65]:
print(
chain.run(df["V_BIOGRAPHY_repo"].loc[30]))
# missing crime tenure

{
    "output": [
        {
            "head": "May 2016",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "organisation of illegal migration",
            "tail_type": "Crime"
        }
    ]
}


In [75]:
df["V_BIOGRAPHY_repo"].loc[33]

' dec 2012 - sentenced by nanhai district court to an undisclosed term of imprisonment for participating in an organised crime gang {1}.'

In [66]:
print(
chain.run(df["V_BIOGRAPHY_repo"].loc[33]))
#wrong tail type

{
    "output": [
        {
            "head": "Dec 2012",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "imprisonment for participating in an organised crime gang",
            "tail_type": "Crime"
        }
    ]
}


In [68]:
print(
chain.run(df["V_BIOGRAPHY_repo"].loc[35]))
# ok - decision contains crime

{
    "output": [
        {
            "head": "Oct 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "settlement fee for alleged violation of listing obligations and disclosure requirements regulations in the matter of dish tv india limited.",
            "tail_type": "Decision"
        }
    ]
}


In [67]:
print(
chain.run(df["V_BIOGRAPHY_repo"].loc[36]))
 # no  judgement present

{
    "output": [
        {
            "head": "Dec 2020",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "violating insurance regulations concerning providing false guarantees with regards to benefits to policyholders",
            "tail_type": "Crime"
        },
        {
            "head": "Dec 2020",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "violating insurance regulations concerning providing false guarantees with regards to benefits to policyholders",
            "tail_type": "Crime"
        }
    ]
}


In [118]:
texts=df.loc[:100,"V_BIOGRAPHY_repo"].to_list()

In [119]:

llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
l3=[]
bio3=[]
ind3=[]
index3=0
for i in texts:
    print(index3)
    
    try:
       output=chain.run(i)
       l3.append(output)
       bio3.append(i)
       ind3.append(index1)
       index3=index3+1
    except:
        index3=index3+1
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [122]:
df_res=pd.DataFrame({})
df_res["Biography"]=bio3
df_res["groups"]=l3
df_res["index"]=ind3

In [123]:
df_res.to_csv("final_res_6.csv")

In [72]:
df_res

,Biography,groups,index
0,mar 2022 - no further information reported. [...,"{\n ""output"": [\n {\n ""he...",0
1,dec 2019 - pleaded not guilty to the criminal...,"{""head"": ""Dec 2019"", ""head_type"": ""Date"", ""rel...",0
2,dec 2018 - reportedly detained. sentenced to ...,"{""head"": ""Dec 2018"", ""head_type"": ""Date"", ""rel...",0
3,jun 2018 - declared as proclaimed offender by...,"{\n ""output"": [\n {\n ""he...",0
4,aug 2013 - declared as proclaimed offender by...,"{\n ""output"": [\n {\n ""he...",0
...,...,...,...
96,jun 2011 - sentenced to an unspecified term o...,"{\n ""output"": [\n {\n ""he...",0
97,mar 2015 - left for syria. 2017 - arrested by...,"{""head"": ""Mar 2015"", ""head_type"": ""Date"", ""rel...",0
98,feb 2016 - arrested by the federal investigat...,"{\n ""output"": [\n {\n ""he...",0
99,jul 2020 - listed by nacta on the 4th schedul...,"{""head"": ""July 2020"", ""head_type"": ""Date"", ""re...",0


## Prompt-4

In [3]:
import random
random.seed(7)
examples_4 = [   {
        "input": 'dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. \
        case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - \
        convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and \
        fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.'\
             , "output":[
        {"head": "July 2019-October 2019",
        "head_type": "Date",
        "relation": "CrimeCommitted",
        "tail": "Bribery",
        "tail_type": "Crime"},{ 
            "head":"Decemeber 2019",
             "head_type":"Date",
             "relation" :"ChargedwithCrime",
             "tail":"Bribery",
             "tail_type":"Crime"},{"head":"March 2022",
             "head_type":"Date",
             "relation" :"Convicted",
             "tail":"Bribery",
             "tail_type":"Crime"}
            
    ,{       "head":"March 2022",
             "head_type":"Date",
             "relation" :"Judgement",
             "tail":"imprisionment,fined",
             "tail_type":"Decision"
        
    }]},
    #year
        {"input":"nov 2021 - expelled from the cpc and dismissed from the\
                    official position for bribery (cny68,400) (2017-2020). case transferred for prosecution.",
         "output": [
           { "head": "2017-2020",
            "head_type": "Date",
            "relation": "CrimeCommitted",
            "tail": "bribery",
            "tail_type": "Crime"},
             {
            "head": "Nov 2021",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "dismissed from the official position for bribery (cny68,400)",
            "tail_type": "Crime"
        },
        {
            "head": "Nov 2021",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "bribery",
            "tail_type": "Crime"
        }

    ]},#year and sentence
     
        {"input":" jun 2022 - pleaded guilty at the magistrate court to the criminal charges of instigating employees in practicing dentistry without \
          authorisation from the brunei medical board (2019 - 2020). fined bnd24,000, in default, 24 months imprisonment.","output": [
        {
            "head": "Jun 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board (2019 - 2020)",
            "tail_type": "Crime"
        },
        {
            "head": "2019 - 2020",
            "head_type": "Date",
            "relation": "CrimeCommited",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board ",
            "tail_type": "Crime"
        }
    ,{   "head": "jun 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined",
            "tail_type": "Decision"}]
},
         

    # year
     {
     "input":' may 2016 - fined rub30,000 by murom city court for organisation of illegal migration (nov 2015 - dec 2015).',
      "output":[{"head":"May 2016","head_type":"Date","relation":"Judgement","tail":"Fined","tial_type":"Decision"},{"head":"November 2015 - December 2015","head_type":"Date","relation":"CrimeCommitted","tail":"illegal migration","tial_type":"Crime"}]
    
     } ,
# year

    {
        "input": 'Oct 2022 - sentenced by Sampit District Court to 5 years \
            imprisonment and fined IDR1b in lieu of additional 3 months imprisonment for methamphetamine trafficking (Feb - 2021). \
            and last year he got an award as the Best Talent'
        ,
        "output":[{"head": "October 2022",
        "head_type": "Date",
        "relation": "Judgement",
        "tail": "sentenced,imprisonment,fined",
        "tail_type": "Decision"},
        {"head":"February-2021",
         "head_type":"Date",
         "relation":"CrimeCommited",
         "tail":"methamphetamine trafficking",
         "tail_type":"Crime"}
    ]} # year and sentence 
  ,

# sentence
   {"input":" jul 2022 - held responsible for company's \
   violation of misrepresentation in financial records. warned and fined cny80,000 by cbirc lianyungang bureau.",
    "output":[
        {
            "head": "Jul 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "held responsible for company's violation of misrepresentation in financial records.",
            "tail_type": "Crime"
        },
        {
            "head": "Jul 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "warned and fined ",
            "tail_type": "Decision"
        }
    ]}
        
    ]



In [4]:
system_prompt ='''You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
Your task is to identify the entities and relations from the input text. You must generate the output as a list containing
JSON objects. Each JSON object should be in format {"head":entity,"head_type":type of entity assigned to head, "tail":entity, "tail_type":type of entity assigned to tail,"relation":relation between entities asigned to head and tail}.
'''

In [5]:
class ExtractedInfo(BaseModel):
    head: str = Field(description="extracted first or head entity like Decmeber-2022 etc.")
    head_type: str = Field(description="type of the extracted head entity like Date etc.")
    relation: str = Field(description="relation between the head and the tail entities or if tail is none then relation be also none.")
    tail: str = Field(description="extracted second or tail entity like Bribery,Fined,Imprisioned, None etc.")
    tail_type: str = Field(description="type of the extracted tail entity like Crime, Sentence, None etc.")
    
parser = JsonOutputParser(pydantic_object=ExtractedInfo)

In [6]:
# all_mrgd_ot_gpu
llm=ChatOllama(model="llama3",temperature=0,format="json")
human_prompt=PromptTemplate(template="""
Below are a list of examples,each having a JSON object with input and output as keys.Input key takes {input}
and output key generates is list of JSON having extracted entity and relations.Generate output as per this format.

examples: {examples}
Use the following relation types, don't use other relation that is not defined below:


For the following input, extract entities and relationships as described in examples.
{format_instructions}
Input text: {input}
In the {input}, identify all date event relationships.
"""
,input_variables=["input"],partial_variables={"examples":examples_4,
        "format_instructions": parser.get_format_instructions()})


human_message_prompt=HumanMessagePromptTemplate(prompt=human_prompt)
# Identify only the relations mentioned in {rel_types} .
# # RELATION TYPES:
# {rel_types}
# "Date" and "Crime"  with relation as "Convicted".Don't provide any other entity and relationship

In [7]:
system_message=SystemMessage(content=system_prompt)
chat_prompt=ChatPromptTemplate.from_messages([system_message,human_message_prompt])

In [8]:


# prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
# chain = chat_prompt | llm

C:\Users\admin\anaconda3\envs\TM_NS\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [18]:
print(
chain.run("oct 2017 - pleaded not guilty to human trafficking (oct 2017). trial scheduled for feb 2018."))
# ok

{
    "input": "oct 2017 - pleaded not guilty to human trafficking (oct 2017). trial scheduled for feb 2018.",
    "output": [
        {
            "head": "Oct 2017",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "human trafficking",
            "tail_type": "Crime"
        },
        {
            "head": "Oct 2017",
            "head_type": "Date",
            "relation": "CrimeCommitted",
            "tail": "human trafficking",
            "tail_type": "Crime"
        },
        {
            "head": "Feb 2018",
            "head_type": "Date",
            "relation": "TrialScheduled",
            "tail": "human trafficking",
            "tail_type": "Crime"
        }
    ]
}


In [19]:
print(
chain.run("feb 2021 - criminally charged by the sessions court with bribery (myr3,000) (dec 2016). pleaded not guilty.\
bail allowed. case re-mention scheduled for mar 2021."))



{
    "output": [
        {
            "head": "Feb 2021",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "Bribery",
            "tail_type": "Crime"
        },
        {
            "head": "Dec 2016",
            "head_type": "Date",
            "relation": "CrimeCommitted",
            "tail": "Bribery",
            "tail_type": "Crime"
        }
    ]
}


In [20]:
print(
chain.run("oct 2022 - criminally charged at the\
magistrate court with cannabis trafficking (5,152g) (sep 2022). no plea recorded. case mention scheduled for dec 2022."))



{
    "output": [
        {
            "head": "Oct 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "Cannabis Trafficking (5,152g)",
            "tail_type": "Crime"
        },
        {
            "head": "Sep 2022",
            "head_type": "Date",
            "relation": "CrimeCommitted",
            "tail": "Cannabis Trafficking (5,152g)",
            "tail_type": "Crime"
        },
        {
            "head": "Dec 2022",
            "head_type": "Date",
            "relation": "CaseMentionScheduled",
            "tail": "None",
            "tail_type": "None"
        }
    ]
}


In [96]:
print(chain.run(" apr 2022 - pleaded guilty at the sessions court to the criminal charges of bribery (myr20,000) (jun 2021). fined myr5,000 in default 1 month imprisonment."))

{"output": [{"head": "Apr 20,22", "head_type": "Date", "relation": "ChargedWithCrime", "tail": "bribery (myr20,000)", "tail_type": "Crime"}, {"head": "Apr 20,22", "head_type": "Date", "relation": "Judgement", "tail": "fined myr5,000 in default 1 month imprisonment", "tail_type": "Decision"}]} 

 
 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 




In [91]:

print(
chain.run("jun 2011 - reportedly under investigation for graft during the distribution of subsidised cooking oil (idr1.3b) (2008). dec 2012 - sentenced to 1 year imprisonment and fined idr50m. jan 2019 - no further information reported."))

{"output": [
    {"head": "jun 2011", "head_type": "Date", "relation": "Under Investigation", "tail": "graft during the distribution of subsidised cooking oil (idr1.3b)", "tail_type": "Crime"},
    {"head": "dec 2012", "head_type": "Date", "relation": "ChargedWithCrime", "tail": "imprisonment and fined idr50m", "tail_type": "Sentence"},
    {"head": "jan 2019", "head_type": "Date", "relation": "No Further Information Reported", "tail": "None", "tail_type": "None"}
]} 

 
 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 




In [92]:
print(
chain.run(" jun 2022 - civil lawsuit filed by the bank pembangunan malaysia (bpmb) for allegedly conspiring and using unlawful means to defraud the development financial institution (dfi) and conceal such fraud and its proceeds from bpmb (myr400m) and bribery (myr8m) (jul 2012). also, reportedly, ex-parte mareva injunction filed by the bpmb. jul 2022 - reportedly an ex-parte mareva injunction obtained from the high court by \
the bpmb to freeze the assets. apr 2023 - reportedly reprimanded and fined myr25,000 by mybursa for breaching market listing requirements (may 2021)."))

{"output": [
    {"head": "Jun 20,22", "head_type": "Date", "relation": "None", "tail": "civil lawsuit filed by the bank pembangunan malaysia (bpmb)", "tail_type": "Event"},
    {"head": "Jul 2012", "head_type": "Date", "relation": "None", "tail": "allegedly conspiring and using unlawful means to defraud the development financial institution (dfi) and conceal such fraud and its proceeds from bpmb (myr400m) and bribery (myr8m)", "tail_type": "Event"},
    {"head": "Jul 20,22", "head_type": "Date", "relation": "None", "tail": "ex-parte mareva injunction obtained from the high court by the bpmb to freeze the assets", "tail_type": "Event"},
    {"head": "Apr 20,23", "head_type": "Date", "relation": "None", "tail": "reprimanded and fined myr25,000 by mybursa for breaching market listing requirements (may 20,21)", "tail_type": "Event"}
]} 

 
 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 




In [95]:
print(chain.run(" apr 2022 - pleaded guilty at the sessions court to the criminal charges \
of bribery (myr20,000) (jun 2021). fined myr5,000 in default 1 month imprisonment."))

{"output": [{"head": "Apr 20,22", "head_type": "Date", "relation": "ChargedWithCrime", "tail": "bribery (myr20,000)", "tail_type": "Crime"}, {"head": "Apr 20,22", "head_type": "Date", "relation": "Judgement", "tail": "fined myr5,000 in default 1 month imprisonment", "tail_type": "Decision"}]} 

 
 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 




In [93]:
print(
chain.run( "accused of participating in a scheme to bribe officials at us citizenship and immigration services to illegally obtain green cards, and officials at the maryland comptroller's office to receive abatement of state taxes (apr 2005 - 2007). sep 2007 - arrested. charged with conspiracy to bribe public officials. sep 2008 - sentenced to 3 years in prison and 3 years supervised release. \
 agreed to forfeit usd323,565. mar 2019 - no further information reported."))

{"output": [
    {"head": "Apr 2005-2007", "head_type": "Date", "relation": "CrimeCommitted", "tail": "bribe officials at us citizenship and immigration services to illegally obtain green cards", "tail_type": "Crime"},
    {"head": "Sep 2007", "head_type": "Date", "relation": "ChargedWithCrime", "tail": "conspiracy to bribe public officials", "tail_type": "Crime"},
    {"head": "Sep 2008", "head_type": "Date", "relation": "Judgement", "tail": "sentenced to 3 years in prison and 3 years supervised release", "tail_type": "Decision"}
]} 

 
 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 




In [94]:
print(
chain.run("jun 2011 - reportedly under investigation for graft during the distribution of subsidised cooking oil (idr1.3b) (2008). \
dec 2012 - sentenced to 1 year imprisonment and fined idr50m. jan 2019 - no further information reported."))

{"output": [
    {"head": "jun 2011", "head_type": "Date", "relation": "Under Investigation", "tail": "graft during the distribution of subsidised cooking oil (idr1.3b)", "tail_type": "Crime"},
    {"head": "dec 2012", "head_type": "Date", "relation": "ChargedWithCrime", "tail": "imprisonment and fined idr50m", "tail_type": "Sentence"},
    {"head": "jan 2019", "head_type": "Date", "relation": "No Further Information Reported", "tail": "None", "tail_type": "None"}
]} 

 
 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 




In [40]:
texts=df.loc[:150,"V_BIOGRAPHY_repo"].to_list()

In [47]:

llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
l4=[]
k=[]
bio4=[]
ind4=[]
index4=0
time4=[]
for i in texts:
       print(index4)
       start4=time.time()
       output=chain.run(i)
       end4=time.time()
       time4.append(end4-start4)
       l4.append(output)

        
       bio4.append(i)
       ind4.append(index4)
       index4=index4+1

        

0
1
2
3


In [38]:
l4

[]

In [15]:
df_res=pd.DataFrame({})
df_res["Biography"]=bio4
df_res["groups"]=l4
df_res["index"]=ind4
df_res["time"]=time4

In [16]:
df_res.to_csv("final_res_7.csv")

In [49]:
df["V_BIOGRAPHY_repo"].loc[35]

' oct 2022 - without admission or denial of guilt, paid joint and several inr4,554,000 to sebi as settlement fee for alleged violation of listing obligations and disclosure requirements regulations in the matter of dish tv india limited.'

## prompt -5

In [43]:
import random
random.seed(7)
examples_5 = [
 #1             
 {"input":"nov 2021 - expelled from the cpc and dismissed from the\
                    official position for bribery (cny68,400) (2017-2020). case transferred for prosecution.",
         "output": [
           { "head": "2017-2020",
            "head_type": "Date",
            "relation": "CrimeCommitted",
            "tail": "bribery",
            "tail_type": "Crime"},
             {
            "head": "Nov 2021",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "dismissed from the official position for bribery (cny68,400)",
            "tail_type": "Decision"
             },
     { "input": 'dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. \
        case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - \
        convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and \
        fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.'\
       ,"output":[
        {"head": "July 2019-October 2019",
        "head_type": "Date",
        "relation": "CrimeCommitted",
        "tail": "Bribery",
        "tail_type": "Crime"},{ 
            "head":"Decemeber 2019",
             "head_type":"Date",
             "relation" :"ChargedwithCrime",
             "tail":"Bribery",
             "tail_type":"Crime"},
           {"head":"March 2022",
             "head_type":"Date",
             "relation" :"Convicted",
             "tail":"Bribery",
             "tail_type":"Crime"}
            
    ,       {"head":"March 2022",
             "head_type":"Date",
             "relation" :"Judgement",
             "tail":" sentenced to 58 years imprisonment and \
        fined myr1.05m in default 72 months imprisonment",
             "tail_type":"Decision" }
       ]},
    #year
       
     

    ]},#year and sentence
     
        {"input":" jun 2022 - pleaded guilty at the magistrate court to the criminal charges of instigating employees in practicing dentistry without \
          authorisation from the brunei medical board (2019 - 2020). fined bnd24,000, in default, 24 months imprisonment.","output": [
        {
            "head": "Jun 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board (2019 - 2020)",
            "tail_type": "Crime"
        },
        {
            "head": "2019 - 2020",
            "head_type": "Date",
            "relation": "CrimeCommited",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board ",
            "tail_type": "Crime"
        }
    ,{   "head": "jun 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined bnd24,000, in default, 24 months imprisonment.",
            "tail_type": "Decision"}]
},
         

    # year
     {
     "input":' may 2016 - fined rub30,000 by murom city court for organisation of illegal migration (nov 2015 - dec 2015).',
      "output":[{"head":"May 2016","head_type":"Date","relation":"Judgement","tail":"fined rub30,000","tial_type":"Decision"},
                {"head":"November 2015 - December 2015","head_type":"Date","relation":"CrimeCommitted","tail":"illegal migration","tial_type":"Crime"}]
    
     } ,
# year

    {
        "input": 'Oct 2022 - sentenced by Sampit District Court to 5 years \
            imprisonment and fined IDR1b in lieu of additional 3 months imprisonment for methamphetamine trafficking (Feb - 2021). \
            and last year he got an award as the Best Talent'
        ,
        "output":[{"head": "October 2022",
        "head_type": "Date",
        "relation": "Judgement",
        "tail": "sentenced by Sampit District Court to 5 years \
            imprisonment and fined IDR1b",
        "tail_type": "Decision"},
        {"head":"February-2021",
         "head_type":"Date",
         "relation":"CrimeCommited",
         "tail":"methamphetamine trafficking",
         "tail_type":"Crime"}
    ]},  # year and sentence 
  
# sentence -16
   {"input":" jul 2022 - held responsible for company's \
   violation of misrepresentation in financial records. warned and fined cny80,000 by cbirc lianyungang bureau.",
    "output":[
        {
            "head": "Jul 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "held responsible for company's violation of misrepresentation in financial records.",
            "tail_type": "Decision"
        },
        {
            "head": "Jul 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "warned and fined ",
            "tail_type": "Decision"
        }
    ]},    {"input":" feb 2023 - reportedly fined bdt0.1m and sentenced to 10 years of \
    imprisonment for trafficking narcotics and seized (0.5m yaba tablets).",
    "output": [
        { #12
            "head": "Feb 2023",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined bdt0.1m and sentenced to 10 years of imprisonment for trafficking narcotics ",
            "tail_type": "Decision"},
        {"head": "Feb 2023",
            "head_type": "Date",
            "relation": "Convicted",
            "tail": "trafficking narcotics and seized (0.5m yaba tablets)",
            "tail_type": "Crime"}]
     }
        
    ]



In [ ]:
' dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and fined myr1.05m in default 72 months imprisonment. \
allowed stay of execution of the sentence pending appeal to the high court.'

In [44]:
system_prompt ='''You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
Your task is to identify the entities and relations from the input text. You must generate the output as a list containing
JSON objects. Each JSON object should be in format {"head":entity,"head_type":type of entity assigned to head, "tail":entity, "tail_type":type of entity assigned to tail,"relation":relation between entities asigned to head and tail}.
'''

In [45]:
class ExtractedInfo(BaseModel):
    head: str = Field(description="extracted first or head entity like Decmeber-2022 etc.")
    head_type: str = Field(description="type of the extracted head entity like Date etc.")
    relation: str = Field(description="relation between the head and the tail entities or if tail is none then relation be also none.")
    tail: str = Field(description="extracted second or tail entity like Bribery,Fined,Imprisioned, None etc.")
    tail_type: str = Field(description="type of the extracted tail entity like Crime, Sentence, None etc.")
    
parser = JsonOutputParser(pydantic_object=ExtractedInfo)

In [47]:
# all_mrgd_ot_gpu
llm=ChatOllama(model="llama3",temperature=0,format="json")
human_prompt=PromptTemplate(template="""
Below are a list of examples,each having a JSON object with input and output as keys.Input key takes {input}
and output key generates is list of JSON having extracted entity and relations.Generate output as per this format.

examples: {examples}
Use the following relation types, don't use other relation that is not defined below:
# RELATION TYPES:
{rel_types}

For the following input, extract entities and relationships as described in examples.
{format_instructions}
Input text: {input}
In the {input}, identify all date event relationships.
Identify only the relations mentioned in {rel_types} .
"""
,input_variables=["input"],partial_variables={"examples":examples_5,"rel_types":["Convicted","CrimeCommited","Judgement","ChargedWithCrime"],
        "format_instructions": parser.get_format_instructions()})


human_message_prompt=HumanMessagePromptTemplate(prompt=human_prompt)


# "Date" and "Crime"  with relation as "Convicted".Don't provide any other entity and relationship

In [48]:
system_message=SystemMessage(content=system_prompt)
chat_prompt=ChatPromptTemplate.from_messages([system_message,human_message_prompt])

In [49]:


# prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
# chain = chat_prompt | llm

In [50]:
df["V_BIOGRAPHY_repo"].loc[1]

' dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.'

In [51]:
 print(
chain.run(df["V_BIOGRAPHY_repo"].loc[1]))

{"head": "Dec 2019", "head_type": "Date", "relation": "ChargedWithCrime", "tail": "pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019)", "tail_type": "Crime"}
 
   


In [52]:
print(
chain.run("oct 2017 - pleaded not guilty to human trafficking (oct 2017). trial scheduled for feb 2018."))
# ok

{
    "output": [
        {
            "head": "Oct 2017",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "human trafficking",
            "tail_type": "Crime"
        },
        {
            "head": "Feb 2018",
            "head_type": "Date",
            "relation": "TrialScheduled",
            "tail": "trial scheduled for feb 2018",
            "tail_type": "Event"
        }
    ]
}


In [53]:
print(
chain.run("feb 2021 - criminally charged by the sessions court with bribery (myr3,000) (dec 2016). pleaded not guilty.\
bail allowed. case re-mention scheduled for mar 2021."))

{
    "output": [
        {
            "head": "Feb 2021",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "bribery (myr3,000)",
            "tail_type": "Crime"
        },
        {
            "head": "Dec 2016",
            "head_type": "Date",
            "relation": "CrimeCommitted",
            "tail": "bribery",
            "tail_type": "Crime"
        }
    ]
}


In [54]:
print(
chain.run("feb 2021 - criminally charged by the sessions court with bribery (myr3,000) (dec 2016). pleaded not guilty.\
bail allowed. case re-mention scheduled for mar 2021."))



{
    "output": [
        {
            "head": "Feb 2021",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "bribery (myr3,000)",
            "tail_type": "Crime"
        },
        {
            "head": "Dec 2016",
            "head_type": "Date",
            "relation": "CrimeCommitted",
            "tail": "bribery",
            "tail_type": "Crime"
        }
    ]
}


In [55]:
print(
chain.run("oct 2022 - criminally charged at the\
magistrate court with cannabis trafficking (5,152g) (sep 2022). no plea recorded. case mention scheduled for dec 2022."))



{
    "output": [
        {
            "head": "Oct 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "cannabis trafficking (5,152g)",
            "tail_type": "Crime"
        },
        {
            "head": "Sep 2022",
            "head_type": "Date",
            "relation": "CrimeCommitted",
            "tail": "cannabis trafficking (5,152g)",
            "tail_type": "Crime"
        }
    ]
}


In [56]:
print(chain.run(" apr 2022 - pleaded guilty at the sessions court to the criminal charges of bribery (myr20,000)\
(jun 2021). fined myr5,000 in default 1 month imprisonment."))

{
    "output": [
        {
            "head": "Apr 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "bribery (myr20,000)",
            "tail_type": "Crime"
        },
        {
            "head": "Apr 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined myr5,000 in default 1 month imprisonment.",
            "tail_type": "Decision"
        }
    ]
}


In [199]:
print(chain.run(" apr 2022 - pleaded guilty at the sessions court to the criminal charges of bribery (myr20,000)\
(jun 2021). fined myr5,000 in default 1 month imprisonment."))

{
    "output": [
        {
            "head": "Apr 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "Bribery (myr20,000)",
            "tail_type": "Crime"
        },
        {
            "head": "Apr 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "Fined myr5,000 in default 1 month imprisonment.",
            "tail_type": "Sentence"
        }
    ]
}


In [200]:

print(
chain.run("jun 2011 - reportedly under investigation for graft during the distribution of subsidised cooking oil (idr1.3b) (2008). dec 2012 - sentenced to 1 year imprisonment and fined idr50m. jan 2019 - no further information reported."))

{
"head": "Jun 2011",
"head_type": "Date",
"relation": "ChargedWithCrime",
"tail": "reportedly under investigation for graft during the distribution of subsidised cooking oil (idr1.3b) (2008)",
"tail_type": "Crime"
}

   


In [201]:
print(
chain.run(" jun 2022 - civil lawsuit filed by the bank pembangunan malaysia (bpmb) for allegedly conspiring and using unlawful means to defraud the development financial institution (dfi) and conceal such fraud and its proceeds from bpmb (myr400m) and bribery (myr8m) (jul 2012). also, reportedly, ex-parte mareva injunction filed by the bpmb. jul 2022 - reportedly an ex-parte mareva injunction obtained from the high court by \
the bpmb to freeze the assets. apr 2023 - reportedly reprimanded and fined myr25,000 by mybursa for breaching market listing requirements (may 2021)."))

{ "head": "Jun 2022", "head_type": "Date", "relation": "ChargedWithCrime", "tail": "civil lawsuit filed by the bank pembangunan malaysia (bpmb) for allegedly conspiring and using unlawful means to defraud the development financial institution (dfi) and conceal such fraud and its proceeds from bpmb (myr400m) and bribery (myr8m) (jul 2012). also, reportedly, ex-parte mareva injunction filed by the bpmb.", "tail_type": "Crime" }
 
   


In [202]:
print(chain.run(" apr 2022 - pleaded guilty at the sessions court to the criminal charges \
of bribery (myr20,000) (jun 2021). fined myr5,000 in default 1 month imprisonment."))

{
    "output": [
        {
            "head": "Apr 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "bribery (myr20,000)",
            "tail_type": "Crime"
        },
        {
            "head": "Apr 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined myr5,000 in default 1 month imprisonment.",
            "tail_type": "Sentence"
        }
    ]
}


In [144]:
print(
chain.run( "accused of participating in a scheme to bribe officials at us citizenship and immigration services to illegally obtain green cards, and officials at the maryland comptroller's office to receive abatement of state taxes (apr 2005 - 2007). sep 2007 - arrested. charged with conspiracy to bribe public officials. sep 2008 - sentenced to 3 years in prison and 3 years supervised release. \
 agreed to forfeit usd323,565. mar 2019 - no further information reported."))

{ "head": "Apr-2005", "head_type": "Date", "relation": "CrimeCommited", "tail": "participating in a scheme to bribe officials at us citizenship and immigration services to illegally obtain green cards, and officials at the maryland comptroller's office to receive abatement of state taxes", "tail_type": "Crime" }
 
 	
   	



   


In [ ]:
    {"input":" feb 2023 - reportedly fined bdt0.1m and sentenced to 10 years of \
    imprisonment for trafficking narcotics and seized (0.5m yaba tablets).",
    "output": [
        { #12
            "head": "Feb 2023",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined bdt0.1m and sentenced to 10 years of imprisonment for trafficking narcotics ",
            "tail_type": "Crime"},
        {"head": "Feb 2023",
            "head_type": "Date",
            "relation": "Convicted",
            "tail": "trafficking narcotics and seized (0.5m yaba tablets)",
            "tail_type": "Crime"}]
     }

In [135]:
import random
random.seed(7)
examples_7 = [ #1
    { "input": 'dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. \
        case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - \
        convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and \
        fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.'\
       ,"output":[
       { 
            "head":"Decemeber 2019",
             "head_type":"Date",
             "relation" :"ChargedwithCrime",
             "tail":"Bribery",
             "tail_type":"Crime"},
             {"head": "July 2019-October 2019",
        "head_type": "Date",
        "relation": "CrimeCommitted",
        "tail": "Bribery",
        "tail_type": "Crime"},
           {"head":"March 2022",
             "head_type":"Date",
             "relation" :"Convicted",
             "tail":"Bribery",
             "tail_type":"Crime"}
            
    ,       {"head":"March 2022",
             "head_type":"Date",
             "relation" :"Judgement",
             "tail":" sentenced to 58 years imprisonment and \
        fined myr1.05m in default 72 months imprisonment",
             "tail_type":"Decision" }
       ]},#35
    {
        "input": 'Oct 2022 - sentenced by Sampit District Court to 5 years \
            imprisonment and fined IDR1b in lieu of additional 3 months imprisonment for methamphetamine trafficking (Feb 2021). \
            and last year he got an award as the Best Talent'
        ,
        "output":[{"head": "October 2022",
        "head_type": "Date",
        "relation": "Judgement",
        "tail": "sentenced by Sampit District Court to 5 years \
            imprisonment and fined IDR1b",
        "tail_type": "Decision"},
        {"head":"February 2021",
         "head_type":"Date",
         "relation":"CrimeCommited",
         "tail":"methamphetamine trafficking",
         "tail_type":"Crime"}
    ]},  #11
    {"input":' 2010 - reportedly declared as proclaimed offender\
  by khairpur court for alleged involvement in kidnapping. aug 2022 - no further information reported.',
   "ouput":[
           {"head": "2010",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "kidnapping",
            "tail_type": "Crime"}
  ]}  #12
   ,
    {"input":" feb 2023 - reportedly fined bdt0.1m and sentenced to 10 years of \
    imprisonment for trafficking narcotics and seized (0.5m yaba tablets).",
    "output": [
        { 
            "head": "Feb 2023",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined bdt0.1m and sentenced to 10 years of imprisonment for trafficking narcotics ",
            "tail_type": "Crime"},
        {"head": "Feb 2023",
            "head_type": "Date",
            "relation": "Convicted",
            "tail": "trafficking narcotics and seized (0.5m yaba tablets)",
            "tail_type": "Crime"}]
     }, #19 
              {"input":" jun 2022 - pleaded guilty at the magistrate court to the criminal charges of instigating employees in practicing dentistry without \
          authorisation from the brunei medical board (2019 - 2020). fined bnd24,000, in default, 24 months imprisonment.",
         "output": [
        {
            "head": "Jun 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board (2019 - 2020)",
            "tail_type": "Crime"
        },
        {
            "head": "2019 - 2020",
            "head_type": "Date",
            "relation": "CrimeCommited",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board ",
            "tail_type": "Crime"
        },
        {   "head": "jun 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined bnd24,000, in default, 24 months imprisonment.",
            "tail_type": "Decision"}]
        },        
     
    #28
        {"input":"nov 2021 - expelled from the cpc and dismissed from the\
                    official position for bribery (cny68,400) (2017-2020). case transferred for prosecution.",
         "output": [
           { "head": "2017-2020",
            "head_type": "Date",
            "relation": "CrimeCommitted",
            "tail": "bribery",
            "tail_type": "Crime"},
             {
            "head": "Nov 2021",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "dismissed from the official position for bribery (cny68,400)",
            "tail_type": "Decision"
             }]},
     
   
       
         
    # 18
        {
            "input":' may 2016 - fined rub30,000 by murom city court for organisation of illegal migration (nov 2015 - dec 2015).',
             "output":[{"head":"May 2016","head_type":"Date","relation":"Judgement","tail":"fined rub30,000","tial_type":"Decision"},
                {"head":"November 2015 - December 2015","head_type":"Date","relation":"CrimeCommitted","tail":"illegal migration","tial_type":"Crime"}]
    
       } ,
# year

    
# 16
   {"input":" jul 2022 - held responsible for company's \
   violation of misrepresentation in financial records. warned and fined cny80,000 by cbirc lianyungang bureau.",
    "output":[
        {
            "head": "Jul 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "held responsible for company's violation of misrepresentation in financial records.",
            "tail_type": "Decision"
        },
        {
            "head": "Jul 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "warned and fined ",
            "tail_type": "Decision"
        }
    ]}
        
    ]



In [127]:
examples_7i=[#19
    {"input":" jun 2022 - pleaded guilty at the magistrate court to the criminal charges of instigating employees in practicing dentistry without \
          authorisation from the brunei medical board (2019 - 2020). fined bnd24,000, in default, 24 months imprisonment.",
         "output": [
        {
            "head": "June 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board (2019 - 2020)",
            "tail_type": "Crime"
        },
        {
            "head": "2019 - 2020",
            "head_type": "Date",
            "relation": "CrimeCommited",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board ",
            "tail_type": "Crime"
        },
        {   "head": "June 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined bnd24,000, in default, 24 months imprisonment.",
            "tail_type": "Decision"}]
        },
    # 16
   {"input":" jul 2022 - held responsible for company's \
   violation of misrepresentation in financial records. warned and fined cny80,000 by cbirc lianyungang bureau.",
    "output":[
        {
            "head": "July 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "held responsible for company's violation of misrepresentation in financial records.",
            "tail_type": "Decision"
        },
        {
            "head": "July 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "warned and fined cny80,000",
            "tail_type": "Decision"
        }
    ]},#35
        {
        "input": 'Oct 2022 - sentenced by Sampit District Court to 5 years \
            imprisonment and fined IDR1b in lieu of additional 3 months imprisonment for methamphetamine trafficking (Feb 2021). \
            and last year he got an award as the Best Talent'
        ,
        "output":[{"head": "October 2022",
        "head_type": "Date",
        "relation": "Judgement",
        "tail": "sentenced by Sampit District Court to 5 years \
            imprisonment and fined IDR1b",
        "tail_type": "Decision"},
        {"head":"February 2021",
         "head_type":"Date",
         "relation":"CrimeCommited",
         "tail":"methamphetamine trafficking",
         "tail_type":"Crime"}
    ]},
    # 1
    {"input":' dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed.\
case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021.mar 2022 - convicted by the sessions \
court on criminal charges of bribery. sentenced to 58 years imprisonment and fined myr1.05m in default 72 months imprisonment. \
allowed stay of execution of the sentence pending appeal to the high court.',\
           "output":[{"head": "December 2019",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "Bribery",
            "tail_type": "Crime"},
            {"head":"July 2019-October 2019",
              "head_type":"Date",
            "relation":"CrimeCommitted",
            "tail":"Bribery",
            "tail_type":"Crime"},
             {"head":"March 2022",
              "head_type":"Date",
            "relation":"Convicted",
            "tail":"Bribery",
            "tail_type":"Crime"},
             {"head":"March 2022",
              "head_type":"Date",
            "relation":"Judgement",
            "tail":"sentenced to 58 years imprisonment and fined myr1.05m in default 72 months imprisonment",
            "tail_type":"Decision"   
             }]},
           
                 #11
    {"input":' 2010 - reportedly declared as proclaimed offender\
  by khairpur court for alleged involvement in kidnapping. aug 2022 - no further information reported.',
   "ouput":[
           {"head": "2010",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "kidnapping",
            "tail_type": "Crime"}]}
           
,
#12
{"input":" feb 2023 - reportedly fined bdt0.1m and sentenced to 10 years of \
    imprisonment for trafficking narcotics and seized (0.5m yaba tablets).",
    "output": [
        { 
            "head": "Feb 2023",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined bdt0.1m and sentenced to 10 years of imprisonment for trafficking narcotics ",
            "tail_type": "Crime"},
        {"head": "Feb 2023",
            "head_type": "Date",
            "relation": "Convicted",
            "tail": "trafficking narcotics and seized (0.5m yaba tablets)",
            "tail_type": "Crime"}]}

]

In [ ]:
examples_9=[

#warned and fined
#19
    {"input":" jun 2022 - pleaded guilty at the magistrate court to the criminal charges of instigating employees in practicing dentistry without \
          authorisation from the brunei medical board (2019 - 2020). fined bnd24,000, in default, 24 months imprisonment.",
         "output": [
        {
            "head": "June 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board (2019 - 2020)",
            "tail_type": "Crime"
        },
        {
            "head": "2019 - 2020",
            "head_type": "Date",
            "relation": "CrimeCommited",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board ",
            "tail_type": "Crime"
        },
        {   "head": "June 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined bnd24,000, in default, 24 months imprisonment.",
            "tail_type": "Decision"}]
        },
#16
       {"input":" jul 2022 - held responsible for company's \
   violation of misrepresentation in financial records. warned and fined cny80,000 by cbirc lianyungang bureau.",
    "output":[
        {
            "head": "July 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "held responsible for company's violation of misrepresentation in financial records.",
            "tail_type": "Decision"
        },
        {
            "head": "July 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "warned and fined cny80,000",
            "tail_type": "Decision"
        },
#12
        {"input":" feb 2023 - reportedly fined bdt0.1m and sentenced to 10 years of \
    imprisonment for trafficking narcotics and seized (0.5m yaba tablets).",
    "output": [
        { 
            "head": "Feb 2023",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined bdt0.1m and sentenced to 10 years of imprisonment for trafficking narcotics ",
            "tail_type": "Crime"},
        {"head": "Feb 2023",
            "head_type": "Date",
            "relation": "Convicted",
            "tail": "trafficking narcotics and seized (0.5m yaba tablets)",
            "tail_type": "Crime"}]}
]


examples_10=[   #date of crime
    #35
        {
        "input": 'Oct 2022 - sentenced by Sampit District Court to 5 years \
            imprisonment and fined IDR1b in lieu of additional 3 months imprisonment for methamphetamine trafficking (Feb 2021). \
            and last year he got an award as the Best Talent'
        ,
        "output":[{"head": "October 2022",
        "head_type": "Date",
        "relation": "Judgement",
        "tail": "sentenced by Sampit District Court to 5 years \
            imprisonment and fined IDR1b",
        "tail_type": "Decision"},
        {"head":"February 2021",
         "head_type":"Date",
         "relation":"CrimeCommited",
         "tail":"methamphetamine trafficking",
         "tail_type":"Crime"}
    ]},
     #28
        {"input":"nov 2021 - expelled from the cpc and dismissed from the\
                    official position for bribery (cny68,400) (2017-2020). case transferred for prosecution.",
         "output": [
             {
            "head": "Nov 2021",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "dismissed from the official position for bribery (cny68,400)",
            "tail_type": "Decision"
             },{ "head": "2017-2020",
            "head_type": "Date",
            "relation": "CrimeCommitted",
            "tail": "bribery",
            "tail_type": "Crime"}]}
]


examples_11=[
  # 1
    {"input":' dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed.\
case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021.mar 2022 - convicted by the sessions \
court on criminal charges of bribery. sentenced to 58 years imprisonment and fined myr1.05m in default 72 months imprisonment. \
allowed stay of execution of the sentence pending appeal to the high court.',\
           "output":[{"head": "December 2019",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "Bribery",
            "tail_type": "Crime"},
            {"head":"July 2019-October 2019",
              "head_type":"Date",
            "relation":"CrimeCommitted",
            "tail":"Bribery",
            "tail_type":"Crime"},
             {"head":"March 2022",
              "head_type":"Date",
            "relation":"Convicted",
            "tail":"Bribery",
            "tail_type":"Crime"},
             {"head":"March 2022",
              "head_type":"Date",
            "relation":"Judgement",
            "tail":"sentenced to 58 years imprisonment and fined myr1.05m in default 72 months imprisonment",
            "tail_type":"Decision"   
             }]},
    
]



In [24]:
# import random

# random.seed(8)
# examples_8 = [
#     {"input":"jan 2009 - sentenced by cbi to 5 years rigorous imprisonment and fined rs 400,000 for \
# fraud, criminal conspiracy and forgery in obtaining illegally a duty drawback of rs 1.3m.","output":[{ "head": "January 2009",
#             "head_type": "Date",
#             "relation": "Judgement",
#             "tail": "sentenced by cbi to 5 years rigorous imprisonment and fined rs 400,000",
#             "tail_type": "Decision"
#         },{"head": "January 2009",
#             "head_type": "Date",
#             "relation": "Convicted",
#             "tail": "fraud, criminal conspiracy and forgery in obtaining illegally a duty drawback of rs 1.3m",
#             "tail_type": "Crime"}]}
#      ,
#     # sentence -16
#    {"input":" jul 2022 - held responsible for company's \
#    violation of misrepresentation in financial records. warned and fined cny80,000 by cbirc lianyungang bureau.",
#     "output":[
#         {
#             "head": "Jul 2022",
#             "head_type": "Date",
#             "relation": "Convicted",
#             "tail": "for company's violation of misrepresentation in financial records.",
#             "tail_type": "Decision"
#         },
#         {
#             "head": "Jul 2022",
#             "head_type": "Date",
#             "relation": "Judgement",
#             "tail": "warned and fined ",
#             "tail_type": "Decision"
#         }
#     ]},
    
#     { "input": 'dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. \
#         case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - \
#         convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and \
#         fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.'\
#        ,"output":[{"input":"jan 2009 - sentenced by cbi to 5 years rigorous imprisonment and fined rs 400,000 for \
# fraud, criminal conspiracy and forgery in obtaining illegally a duty drawback of rs 1.3m.","output":[{ "head": "January 2009",
#             "head_type": "Date",
#             "relation": "Judgement",
#             "tail": "sentenced by cbi to 5 years rigorous imprisonment and fined rs 400,000",
#             "tail_type": "Decision"
#         },{"head": "January 2009",
#             "head_type": "Date",
#             "relation": "Convicted",
#             "tail": "fraud, criminal conspiracy and forgery in obtaining illegally a duty drawback of rs 1.3m",
#             "tail_type": "Crime"}]}
#      ,
#        { 
#             "head":"Decemeber 2019",
#              "head_type":"Date",
#              "relation" :"ChargedwithCrime",
#              "tail":"Bribery",
#              "tail_type":"Crime"},
#              {"head": "July 2019-October 2019",
#         "head_type": "Date",
#         "relation": "CrimeCommitted",
#         "tail": "Bribery",
#         "tail_type": "Crime"},
#            {"head":"March 2022",
#              "head_type":"Date",
#              "relation" :"Convicted",
#              "tail":"Bribery",
#              "tail_type":"Crime"}
            
#     ,       {"head":"March 2022",
#              "head_type":"Date",
#              "relation" :"Judgement",
#              "tail":" sentenced to 58 years imprisonment and \
#         fined myr1.05m in default 72 months imprisonment",
#              "tail_type":"Decision" }
#        ]},{
#         "input": 'Oct 2022 - sentenced by Sampit District Court to 5 years \
#             imprisonment and fined IDR1b in lieu of additional 3 months imprisonment for methamphetamine trafficking (Feb - 2021). \
#             and last year he got an award as the Best Talent'
#         ,
#         "output":[{"head": "October 2022",
#         "head_type": "Date",
#         "relation": "Judgement",
#         "tail": "sentenced by Sampit District Court to 5 years \
#             imprisonment and fined IDR1b",
#         "tail_type": "Decision"},
#         {"head":"February-2021",
#          "head_type":"Date",
#          "relation":"CrimeCommited",
#          "tail":"methamphetamine trafficking",
#          "tail_type":"Crime"}
#     ]},  {"input":' 2010 - reportedly declared as proclaimed offender\
#   by khairpur court for alleged involvement in kidnapping. aug 2022 - no further information reported.',
#    "ouput":[
#            {"head": "2010",
#             "head_type": "Date",
#             "relation": "ChargedWithCrime",
#             "tail": "kidnapping",
#             "tail_type": "Crime"}
#   ]}
#    ,
#     {"input":" feb 2023 - reportedly fined bdt0.1m and sentenced to 10 years of \
#     imprisonment for trafficking narcotics and seized (0.5m yaba tablets).",
#     "output": [
#         { #12
#             "head": "Feb 2023",
#             "head_type": "Date",
#             "relation": "Judgement",
#             "tail": "fined bdt0.1m and sentenced to 10 years of imprisonment for trafficking narcotics ",
#             "tail_type": "Crime"},
#         {"head": "Feb 2023",
#             "head_type": "Date",
#             "relation": "Convicted",
#             "tail": "trafficking narcotics and seized (0.5m yaba tablets)",
#             "tail_type": "Crime"}]
#      }, #1 
#               {"input":" jun 2022 - pleaded guilty at the magistrate court to the criminal charges of instigating employees in practicing dentistry without \
#           authorisation from the brunei medical board (2019 - 2020). fined bnd24,000, in default, 24 months imprisonment.",
#          "output": [
#         {
#             "head": "Jun 2022",
#             "head_type": "Date",
#             "relation": "ChargedWithCrime",
#             "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board (2019 - 2020)",
#             "tail_type": "Crime"
#         },
#         {
#             "head": "2019 - 2020",
#             "head_type": "Date",
#             "relation": "CrimeCommited",
#             "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board ",
#             "tail_type": "Crime"
#         },
#         {   "head": "jun 2022",
#             "head_type": "Date",
#             "relation": "Judgement",
#             "tail": "fined bnd24,000, in default, 24 months imprisonment.",
#             "tail_type": "Decision"}]
#         },        
     
#     #year
#         {"input":"nov 2021 - expelled from the cpc and dismissed from the\
#                     official position for bribery (cny68,400) (2017-2020). case transferred for prosecution.",
#          "output": [
#            { "head": "2017-2020",
#             "head_type": "Date",
#             "relation": "CrimeCommitted",
#             "tail": "bribery",
#             "tail_type": "Crime"},
#              {
#             "head": "Nov 2021",
#             "head_type": "Date",
#             "relation": "Judgement",
#             "tail": "dismissed from the official position for bribery (cny68,400)",
#             "tail_type": "Decision"
#              }]},
#   # year and sentence 

     
       
         
#     # year
#         {
#             "input":' may 2016 - fined rub30,000 by murom city court for organisation of illegal migration (nov 2015 - dec 2015).',
#              "output":[{"head":"May 2016","head_type":"Date","relation":"Judgement","tail":"fined rub30,000","tial_type":"Decision"},
#                 {"head":"November 2015 - December 2015","head_type":"Date","relation":"CrimeCommitted","tail":"illegal migration","tial_type":"Crime"}]
    
#        } ,
# # year

    

        
#     ]



In [136]:
system_prompt ='''You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
Your task is to identify the entities and relations from the input text. You must generate the output as a list containing
JSON objects. Each JSON object should be in format {"head":entity,"head_type":type of entity assigned to head, "tail":entity, "tail_type":type of entity assigned to tail,"relation":relation between entities asigned to head and tail}.
'''

In [137]:
class ExtractedInfo(BaseModel):
    head: str = Field(description="extracted first or head entity like Decmeber-2022 etc.")
    head_type: str = Field(description="type of the extracted head entity like Date etc.")
    relation: str = Field(description="relation between the head and the tail entities or if tail is none then relation be also none.")
    tail: str = Field(description="extracted second or tail entity like Bribery,Fined,Imprisioned, None etc.")
    tail_type: str = Field(description="type of the extracted tail entity like Crime, Sentence, None etc.")
    
parser = JsonOutputParser(pydantic_object=ExtractedInfo)

In [152]:
# all_mrgd_ot_gpu
llm=ChatOllama(model="llama3",temperature=0,format="json")
human_prompt=PromptTemplate(template="""
Below are a list of examples,each having a JSON object with input and output as keys.Input key takes {input}
and output key generates is list of JSON having extracted entity and relations.Generate output as per this format.

examples: {examples}
Use the following relation types, don't use other relation that is not defined below:
# RELATION TYPES:
{rel_types}

For the following input, extract entities and relationships as described in examples.
{format_instructions}
Input text: {input}
From the {input}, extract all date event relations in output.
Provide only the relations mentioned in {rel_types} .
"""
,input_variables=["input"],partial_variables={"examples":examples_7i,"rel_types":["Convicted","CrimeCommited","Judgement","ChargedWithCrime"],
        "format_instructions": parser.get_format_instructions()})


human_message_prompt=HumanMessagePromptTemplate(prompt=human_prompt)


# "Date" and "Crime"  with relation as "Convicted".Don't provide any other entity and relationship

In [153]:
system_message=SystemMessage(content=system_prompt)
chat_prompt=ChatPromptTemplate.from_messages([system_message,human_message_prompt])

In [154]:


# prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
# chain = chat_prompt | llm

In [145]:
df["V_BIOGRAPHY_repo"].loc[1]

' dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.'

In [146]:
 print(
chain.run(df["V_BIOGRAPHY_repo"].loc[1]))

{
"head": "Dec 2019",
"head_type": "Date",
"relation": "CrimeCommitted",
"tail": "bribery (myr295,000)",
"tail_type": "Crime"
}
   


In [123]:
print(df["V_BIOGRAPHY_repo"].loc[6])

 jan 2009 - sentenced by cbi to 5 years rigorous imprisonment and fined rs 400,000 for fraud, criminal conspiracy and forgery in obtaining illegally a duty drawback of rs 1.3m.


In [124]:
print(df["V_BIOGRAPHY_repo"].loc[5])
print(
chain.run(df["V_BIOGRAPHY_repo"].loc[5]))

print(df["V_BIOGRAPHY_repo"].loc[6])
print(
chain.run(df["V_BIOGRAPHY_repo"].loc[6]))

 dec 2011 - held responsible for misrepresentation in business records. warned and fined cny20,000 by cbirc hubei bureau. jun 2022 - no further information reported.
{
    "output": [
        {
            "head": "Dec 2011",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "misrepresentation in business records",
            "tail_type": "Crime"
        }
    ]
}
 jan 2009 - sentenced by cbi to 5 years rigorous imprisonment and fined rs 400,000 for fraud, criminal conspiracy and forgery in obtaining illegally a duty drawback of rs 1.3m.
{
    "output": [
        {
            "head": "Jan 2009",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "fraud, criminal conspiracy and forgery in obtaining illegally a duty drawback of rs 1.3m.",
            "tail_type": "Crime"
        }
    ]
}


In [125]:
print(df["V_BIOGRAPHY_repo"].loc[19])
print(
chain.run(df["V_BIOGRAPHY_repo"].loc[19]))

 jun 2022 - pleaded guilty at the magistrate court to the criminal charges of instigating employees in practicing dentistry without authorisation from the brunei medical board (2019 - 2020). fined bnd24,000, in default, 24 months imprisonment.
{
    "output": [
        {
            "head": "Jun 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board (2019 - 2020)",
            "tail_type": "Crime"
        },
        {
            "head": "2019 - 2020",
            "head_type": "Date",
            "relation": "CrimeCommited",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board ",
            "tail_type": "Crime"
        },
        {
            "head": "Jun 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined bnd24,000, in de

In [126]:
print(
chain.run("oct 2017 - pleaded not guilty to human trafficking (oct 2017). trial scheduled for feb 2018."))
# ok

{
    "input": "oct 2017 - pleaded not guilty to human trafficking (oct 2017). trial scheduled for feb 2018.",
    "output": [
        {
            "head": "Oct 2017",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "human trafficking",
            "tail_type": "Crime"
        },
        {
            "head": "Oct 2017",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "pleaded not guilty",
            "tail_type": "Decision"
        },
        {
            "head": "Feb 2018",
            "head_type": "Date",
            "relation": "TrialScheduled",
            "tail": "trial scheduled for feb 2018",
            "tail_type": "Event"
        }
    ]
}


In [132]:
print(
chain.run("feb 2021 - criminally charged by the sessions court with bribery (myr3,000) (dec 2016). pleaded not guilty.\
bail allowed. case re-mention scheduled for mar 2021."))

{
    "output": [
        {
            "head": "Feb 2021",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "bribery (myr3,000)",
            "tail_type": "Crime"
        },
        {
            "head": "Dec 2016",
            "head_type": "Date",
            "relation": "CrimeCommited",
            "tail": "bribery (myr3,000)",
            "tail_type": "Crime"
        }
    ]
}


In [133]:
print(
chain.run("feb 2021 - criminally charged by the sessions court with bribery (myr3,000) (dec 2016). pleaded not guilty.\
bail allowed. case re-mention scheduled for mar 2021."))



{
    "output": [
        {
            "head": "Feb 2021",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "bribery (myr3,000)",
            "tail_type": "Crime"
        }
    ]
}


In [134]:
print(
chain.run("oct 2022 - criminally charged at the magistrate court with cannabis trafficking (5,152g) (sep 2022). no plea recorded. case mention scheduled for dec 2022."))



{
    "output": [
        {
            "head": "Oct 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "cannabis trafficking (5,152g)",
            "tail_type": "Crime"
        },
        {
            "head": "Oct 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "no plea recorded.",
            "tail_type": "Decision"
        },
        {
            "head": "Dec 2022",
            "head_type": "Date",
            "relation": "CaseMentionScheduled",
            "tail": "case mention scheduled for dec 2022.",
            "tail_type": "Event"
        }
    ]
}


In [69]:
print(chain.run(" apr 2022 - pleaded guilty at the sessions court to the criminal charges of bribery (myr20,000)\
(jun 2021). fined myr5,000 in default 1 month imprisonment."))

{
    "output": [
        {
            "head": "Apr 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "Bribery",
            "tail_type": "Crime"
        },
        {
            "head": "Jun 2021",
            "head_type": "Date",
            "relation": "CrimeCommitted",
            "tail": "Bribery",
            "tail_type": "Crime"
        },
        {
            "head": "Apr 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "Fined myr5,000 in default 1 month imprisonment.",
            "tail_type": "Sentence"
        }
    ]
}


In [70]:

print(
chain.run("jun 2011 - reportedly under investigation for graft during the distribution of subsidised cooking oil (idr1.3b) (2008). dec 2012 - sentenced to 1 year imprisonment and fined idr50m. jan 2019 - no further information reported."))

{
"head": "jun 2011",
"head_type": "Date",
"relation": "CrimeCommitted",
"tail": "graft during the distribution of subsidised cooking oil (idr1.3b)",
"tail_type": "Crime"
}
   


In [71]:
print(
chain.run(" jun 2022 - civil lawsuit filed by the bank pembangunan malaysia (bpmb) for allegedly conspiring and using unlawful means to\
defraud the development financial institution (dfi) and conceal such fraud and its proceeds from bpmb (myr400m) and bribery (myr8m) (jul 2012).\
also, reportedly, ex-parte mareva injunction filed by the bpmb. jul 2022 - reportedly an ex-parte mareva injunction obtained from the high court by 
the bpmb to freeze the assets. apr 2023 - reportedly reprimanded and fined myr25,000 by mybursa for breaching market listing requirements (may 2021)."))

{"input": "jun 2022 - civil lawsuit filed by the bank pembangunan malaysia (bpmb) for allegedly conspiring and using unlawful means to defraud the development financial institution (dfi) and conceal such fraud and its proceeds from bpmb (myr400m) and bribery (myr8m) (jul 2012). also, reportedly, ex-parte mareva injunction filed by the bpmb. jul 2022 - reportedly an ex-parte mareva injunction obtained from the high court by the bpmb to freeze the assets. apr 2023 - reportedly reprimanded and fined myr25,000 by mybursa for breaching market listing requirements (may 2021).", "output": [{"head": "Jun 2022", "head_type": "Date", "relation": "ChargedWithCrime", "tail": "allegedly conspiring and using unlawful means to defraud the development financial institution (dfi) and conceal such fraud and its proceeds from bpmb (myr400m) and bribery (myr8m)", "tail_type": "Crime"}, {"head": "Jul 2022", "head_type": "Date", "relation": "ChargedWithCrime", "tail": "ex-parte mareva injunction filed by th

In [72]:
print(
chain.run( "accused of participating in a scheme to bribe officials at us citizenship and immigration services to illegally obtain green cards, and officials at the maryland comptroller's office to receive abatement of state taxes (apr 2005 - 2007). sep 2007 - arrested. charged with conspiracy to bribe public officials. sep 2008 - sentenced to 3 years in prison and 3 years supervised release. \
 agreed to forfeit usd323,565. mar 2019 - no further information reported."))

{
"head": "Apr 2005-2007",
"head_type": "Date",
"relation": "CrimeCommitted",
"tail": "scheme to bribe officials at us citizenship and immigration services to illegally obtain green cards, and officials at the maryland comptroller's office to receive abatement of state taxes",
"tail_type": "Crime"
}

   


In [40]:
print(
chain.run("jun 2011 - reportedly under investigation for graft during the distribution of subsidised cooking oil (idr1.3b) (2008). \
dec 2012 - sentenced to 1 year imprisonment and fined idr50m. jan 2019 - no further information reported."))

{
    "head": "Jun 2011",
    "head_type": "Date",
    "relation": "ChargedWithCrime",
    "tail": "graft during the distribution of subsidised cooking oil (idr1.3b)",
    "tail_type": "Crime"
}
   


In [147]:
texts=df.loc[0:150,"V_BIOGRAPHY_repo"].to_list()

In [ ]:

llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
l5=[]
bio5=[]
ind5=[]
index5=0
time5=[]
for i in texts:
    print(index5)
    
    try:
       start5=time.time()
       output=chain.run(i)
       end5=time.time()
       time5.append(end5-start5)
       l5.append(output)
       bio5.append(i)
       ind5.append(index5)
       index5=index5+1
    except:
        time5.append(0)
        index5=index5+1
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113


In [ ]:
df_res=pd.DataFrame({})
df_res["Biography"]=bio5
df_res["groups"]=l5
df_res["index"]=ind5


In [150]:
df_res["time"]=time5

In [ ]:
df_res.to_csv("final_res_ex_7i.csv")

In [27]:
l5

NameError: name 'l5' is not defined

## prompt -6

In [3]:
import random
random.seed(7)
examples_6 = [ {"input":" may 2012 - sentenced by xihu district court to 3 years imprisonment for extortion (cny6.57m) (sep 2011).",
                "output":[{"head":"May 2012","relation":"Judgement","tail":"sentenced by xihu district court to 3 years imprisonment","tail_type":"Decision"\
                          },{"head":"May 2012","relation":"Convicted","tail":"extortion","tail_type":"Crime"},
                          {"head":"September 2011","head_type":"Date","relation":"CrimeCommitted","tail":"extortion","tail_type":"Crime"}]},
#19
            {"input":" jun 2022 - pleaded guilty at the magistrate court to the criminal charges of instigating employees in practicing dentistry without \
          authorisation from the brunei medical board (2019 - 2020). fined bnd24,000, in default, 24 months imprisonment.","output": [
        {
            "head": "June 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board (2019 - 2020)",
            "tail_type": "Crime"
        },
        {
            "head": "2019 - 2020",
            "head_type": "Date",
            "relation": "CrimeCommited",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board ",
            "tail_type": "Crime"
        }
    ,{   "head": "jun 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined",
            "tail_type": "Decision"}]
},
    {"input":" feb 2023 - reportedly fined bdt0.1m and sentenced to 10 years of \
    imprisonment for trafficking narcotics and seized (0.5m yaba tablets).",
    "output": [
        {
            "head": "Feb 2023",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined bdt0.1m and sentenced to 10 years of imprisonment for trafficking narcotics ",
            "tail_type": "Decision"
        },{"head": "Feb 2023",
            "head_type": "Date",
            "relation": "Convicted",
            "tail": "trafficking narcotics and seized (0.5m yaba tablets)",
            "tail_type": "Crime"}
    ]
}, {
        "input": 'Oct 2022 - sentenced by Sampit District Court to 5 years \
            imprisonment and fined IDR1b in lieu of additional 3 months imprisonment for methamphetamine trafficking (Feb - 2021). \
            and last year he got an award as the Best Talent'
        ,
        "output":[{"head": "October 2022",
        "head_type": "Date",
        "relation": "Judgement",
        "tail": "sentenced by Sampit District Court to 5 years imprisonment and fined IDR1b",
        "tail_type": "Decision"},
        {"head":"February 2021",
         "head_type":"Date",
         "relation":"CrimeCommited",
         "tail":"methamphetamine trafficking",
         "tail_type":"Crime"}
    ]},
              {
        "input": 'dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. \
        case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - \
        convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and \
        fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.'\
             , "output":[
        {"head": "July 2019-October 2019",
        "head_type": "Date",
        "relation": "CrimeCommitted",
        "tail": "Bribery",
        "tail_type": "Crime"},{ 
            "head":"Decemeber 2019",
             "head_type":"Date",
             "relation" :"ChargedwithCrime",
             "tail":"Bribery",
             "tail_type":"Crime"},{"head":"March 2022",
             "head_type":"Date",
             "relation" :"Convicted",
             "tail":"Bribery",
             "tail_type":"Crime"}
            
    ,{       "head":"March 2022",
             "head_type":"Date",
             "relation" :"Judgement",
             "tail":"sentenced to 58 years imprisonment and \
        fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.",
             "tail_type":"Decision"
        
    }]},
    
    #year
        {"input":"nov 2021 - expelled from the cpc and dismissed from the\
                    official position for bribery (cny68,400) (2017-2020). case transferred for prosecution.",
         "output": [
           { "head": "2017-2020",
            "head_type": "Date",
            "relation": "CrimeCommitted",
            "tail": "bribery",
            "tail_type": "Crime"},
             {
            "head": "Nov 2021",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "dismissed from the official position for bribery (cny68,400)",
            "tail_type": "Crime"
        },
        {
            "head": "Nov 2021",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "bribery",
            "tail_type": "Crime"
        }

    ]},#year and sentence
     

         

    # year
     {
     "input":' may 2016 - fined rub30,000 by murom city court for organisation of illegal migration (nov 2015 - dec 2015).',
      "output":[{"head":"May 2016","head_type":"Date","relation":"Judgement","tail":"Fined","tial_type":"Decision"},{"head":"November 2015 - December 2015","head_type":"Date","relation":"CrimeCommitted","tail":"illegal migration","tial_type":"Crime"}]
    
     } 
# year

  # year and sentence 
  
# sentence

        
    ]



In [4]:
system_prompt ='''You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
Your task is to identify the entities and relations from the input text. You must generate the output as a list containing
JSON objects. Each JSON object should be in format {"head":entity,"head_type":type of entity assigned to head, "tail":entity, "tail_type":type of entity assigned to tail,"relation":relation between entities asigned to head and tail}.
'''

In [5]:
class ExtractedInfo(BaseModel):
    head: str = Field(description="extracted first or head entity like Decmeber-2022 etc.")
    head_type: str = Field(description="type of the extracted head entity like Date etc.")
    relation: str = Field(description="relation between the head and the tail entities or if tail is none then relation be also none.")
    tail: str = Field(description="extracted second or tail entity like Bribery,Fined,Imprisioned, None etc.")
    tail_type: str = Field(description="type of the extracted tail entity like Crime, Sentence, None etc.")
    
parser = JsonOutputParser(pydantic_object=ExtractedInfo)

In [6]:
# all_mrgd_ot_gpu
llm=ChatOllama(model="llama3",temperature=0,format="json")
human_prompt=PromptTemplate(template="""
Below are a list of examples,each having a JSON object with input and output as keys.Input key takes {input}
and output key generates is list of JSON having extracted entity and relations.Generate output as per this format.

examples: {examples}
Use the following relation types, don't use other relation that is not defined below:
# RELATION TYPES:
{rel_types}

For the following input, extract entities and relationships as described in examples.
{format_instructions}
Input text: {input}
In the {input}, identify all date event relationships.

"""
,input_variables=["input"],partial_variables={"examples":examples_6,"rel_types":["Convicted","CrimeCommited","Judgement","ChargedWithCrime"],
        "format_instructions": parser.get_format_instructions()})


human_message_prompt=HumanMessagePromptTemplate(prompt=human_prompt)


# "Date" and "Crime"  with relation as "Convicted".Don't provide any other entity and relationship

In [7]:
system_message=SystemMessage(content=system_prompt)
chat_prompt=ChatPromptTemplate.from_messages([system_message,human_message_prompt])

In [8]:


# prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
# chain = chat_prompt | llm

C:\Users\admin\anaconda3\envs\TM_NS\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [9]:
df["V_BIOGRAPHY_repo"].loc[23]

' 2010 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. aug 2022 - no further information reported.'

In [10]:
 print(
chain.run(df["V_BIOGRAPHY_repo"].loc[23]))

C:\Users\admin\anaconda3\envs\TM_NS\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{
    "input": {
        "2010 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. aug 2022 - no further information reported."
   	: [
            {"head": "2010", "head_type": "Date", "relation": "ChargedWithCrime", "tail": "alleged involvement in kidnapping", "tail_type": "Crime"},
            {"head": "Aug 2022", "head_type": "Date", "relation": "No Further Information", "tail": "no further information reported", "tail_type": "None"}
        ]
    }
}


In [17]:
texts=df.loc[0:150,"V_BIOGRAPHY_repo"].to_list()

In [11]:

llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
l6=[]
bio6=[]
ind6=[]
index6=0
time6=[]
for i in texts:
    print(index6)
    
    try:
       start6=time.time()
       output=chain.run(i)
       end6=time.time()
       time6.append(end6-start6)
       l6.append(output)
       bio6.append(i)
       ind6.append(index6)
       index6=index6+1
    except:
        time6.append(0)
        index6=index6+1
        

0


C:\Users\admin\anaconda3\envs\TM_NS\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150


In [12]:
df_res=pd.DataFrame({})
df_res["Biography"]=bio6
df_res["groups"]=l6
df_res["index"]=ind6


In [13]:
df_res["time"]=time6

In [14]:
df_res

,Biography,groups,index,time
0,mar 2022 - no further information reported. [...,"{\n ""output"": [\n {\n ""he...",0,22.686620
1,dec 2019 - pleaded not guilty to the criminal...,"{""head"": ""July 2019-October 2019"", ""head_type""...",1,5.134090
2,dec 2018 - reportedly detained. sentenced to ...,"{""head"": ""Dec 2018"", ""head_type"": ""Date"", ""rel...",2,5.306748
3,jun 2018 - declared as proclaimed offender by...,"{\n ""input"": [\n {\n ""hea...",3,8.316620
4,aug 2013 - declared as proclaimed offender by...,"{\n ""input"": [\n {\n ""hea...",4,8.624932
...,...,...,...,...
146,nov 2017 - detained by pakistan rangers sindh...,"{""head"": ""Nov 2017"", ""head_type"": ""Date"", ""rel...",146,5.258737
147,apr 2014 - sentenced by baiyin municipal inte...,"{\n ""output"": [\n {\n ""he...",147,9.568724
148,may 2009 - reportedly criminally charged with...,"{""head"": ""May 2009"", ""head_type"": ""Date"", ""rel...",148,5.373075
149,may 2022 - convicted of income tax and goods ...,"{ ""head"": ""May 2022"", ""head_type"": ""Date"", ""re...",149,5.309731


In [15]:
df_res.to_csv("final_res_ex_6.csv")

## Prompt 6i

In [16]:
import random
random.seed(7)
examples_6i = [ {"input":" may 2012 - sentenced by xihu district court to 3 years imprisonment for extortion (cny6.57m) (sep 2011).",
                "output":[{"head":"May 2012","relation":"Judgement","tail":"sentenced by xihu district court to 3 years imprisonment","tail_type":"Decision"\
                          },{"head":"May 2012","relation":"Convicted","tail":"extortion","tail_type":"Crime"},
                          {"head":"September 2011","head_type":"Date","relation":"CrimeCommitted","tail":"extortion","tail_type":"Crime"}]},
#19
            {"input":" jun 2022 - pleaded guilty at the magistrate court to the criminal charges of instigating employees in practicing dentistry without \
          authorisation from the brunei medical board (2019 - 2020). fined bnd24,000, in default, 24 months imprisonment.","output": [
        {
            "head": "June 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board",
            "tail_type": "Crime"
        },
        {
            "head": "2019 - 2020",
            "head_type": "Date",
            "relation": "CrimeCommited",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board ",
            "tail_type": "Crime"
        }
    ,{   "head": "June 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined",
            "tail_type": "Decision"}]
},
    {"input":" feb 2023 - reportedly fined bdt0.1m and sentenced to 10 years of \
    imprisonment for trafficking narcotics and seized (0.5m yaba tablets).",
    "output": [
        {
            "head": "February 2023",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined bdt0.1m and sentenced to 10 years of imprisonment for trafficking narcotics ",
            "tail_type": "Decision"
        },{"head": "February 2023",
            "head_type": "Date",
            "relation": "Convicted",
            "tail": "trafficking narcotics",
            "tail_type": "Crime"}
    ]
}, {
        "input": 'Oct 2022 - sentenced by Sampit District Court to 5 years \
            imprisonment and fined IDR1b in lieu of additional 3 months imprisonment for methamphetamine trafficking (Feb - 2021). \
            and last year he got an award as the Best Talent'
        ,
        "output":[{"head": "October 2022",
        "head_type": "Date",
        "relation": "Judgement",
        "tail": "sentenced by Sampit District Court to 5 years imprisonment and fined IDR1b",
        "tail_type": "Decision"},
        {"head":"February 2021",
         "head_type":"Date",
         "relation":"CrimeCommited",
         "tail":"methamphetamine trafficking",
         "tail_type":"Crime"}
    ]},
              {
        "input": 'dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. \
        case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - \
        convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and \
        fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.'\
             , "output":[
        {"head": "July 2019-October 2019",
        "head_type": "Date",
        "relation": "CrimeCommitted",
        "tail": "Bribery",
        "tail_type": "Crime"},{ 
            "head":"Decemeber 2019",
             "head_type":"Date",
             "relation" :"ChargedwithCrime",
             "tail":"Bribery",
             "tail_type":"Crime"},{"head":"March 2022",
             "head_type":"Date",
             "relation" :"Convicted",
             "tail":"Bribery",
             "tail_type":"Crime"}
            
    ,{       "head":"March 2022",
             "head_type":"Date",
             "relation" :"Judgement",
             "tail":"sentenced to 58 years imprisonment and \
        fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.",
             "tail_type":"Decision"
        
    }]},
    
    #year
        {"input":"nov 2021 - expelled from the cpc and dismissed from the\
                    official position for bribery (cny68,400) (2017-2020). case transferred for prosecution.",
         "output": [
           { "head": "2017-2020",
            "head_type": "Date",
            "relation": "CrimeCommitted",
            "tail": "bribery",
            "tail_type": "Crime"},
             {
            "head": "November 2021",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "dismissed from the official position for bribery (cny68,400)",
            "tail_type": "Decision"
        },
        {
            "head": "November  2021",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "bribery",
            "tail_type": "Crime"
        }

    ]},#year and sentence
     

         

    # year
     {
     "input":' may 2016 - fined rub30,000 by murom city court for organisation of illegal migration (nov 2015 - dec 2015).',
      "output":[{"head":"May 2016","head_type":"Date","relation":"Judgement","tail":"Fined rub30,000","tial_type":"Decision"},{"head":"November 2015 - December 2015","head_type":"Date","relation":"CrimeCommitted","tail":"illegal migration","tial_type":"Crime"}]
    
     } 
# year

  # year and sentence 
  
# sentence

        
    ]



In [ ]:
import random
random.seed(7)
examples_6ii = [ {"input":" may 2012 - sentenced by xihu district court to 3 years imprisonment for extortion (cny6.57m) (sep 2011).",
                "output":[{"head":"May 2012","relation":"Judgement","tail":"sentenced by xihu district court to 3 years imprisonment","tail_type":"Decision"\
                          },{"head":"May 2012","relation":"Convicted","tail":"extortion","tail_type":"Crime"},
                          {"head":"September 2011","head_type":"Date","relation":"CrimeCommitted","tail":"extortion","tail_type":"Crime"}]},
#19
            {"input":" jun 2022 - pleaded guilty at the magistrate court to the criminal charges of instigating employees in practicing dentistry without \
          authorisation from the brunei medical board (2019 - 2020). fined bnd24,000, in default, 24 months imprisonment.","output": [
        {
            "head": "June 2022",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board",
            "tail_type": "Crime"
        },
        {
            "head": "2019 - 2020",
            "head_type": "Date",
            "relation": "CrimeCommited",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board ",
            "tail_type": "Crime"
        }
    ,{   "head": "June 2022",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined bnd24,000",
            "tail_type": "Decision"}]
},
    {"input":" feb 2023 - reportedly fined bdt0.1m and sentenced to 10 years of \
    imprisonment for trafficking narcotics and seized (0.5m yaba tablets).",
    "output": [
        {
            "head": "February 2023",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "fined bdt0.1m and sentenced to 10 years of imprisonment for trafficking narcotics ",
            "tail_type": "Decision"
        },{"head": "February 2023",
            "head_type": "Date",
            "relation": "Convicted",
            "tail": "trafficking narcotics",
            "tail_type": "Crime"}
    ]
}, {
        "input": 'Oct 2022 - sentenced by Sampit District Court to 5 years \
            imprisonment and fined IDR1b in lieu of additional 3 months imprisonment for methamphetamine trafficking (Feb - 2021). \
            and last year he got an award as the Best Talent'
        ,
        "output":[{"head": "October 2022",
        "head_type": "Date",
        "relation": "Judgement",
        "tail": "sentenced by Sampit District Court to 5 years imprisonment and fined IDR1b in lieu of additional 3 months imprisonment ",
        "tail_type": "Decision"},
        {"head":"February 2021",
         "head_type":"Date",
         "relation":"CrimeCommited",
         "tail":"methamphetamine trafficking",
         "tail_type":"Crime"}
    ]},
              {
        "input": 'dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. \
        case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - \
        convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and \
        fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.'\
             , "output":[
        {"head": "July 2019-October 2019",
        "head_type": "Date",
        "relation": "CrimeCommitted",
        "tail": "Bribery",
        "tail_type": "Crime"},{ 
            "head":"Decemeber 2019",
             "head_type":"Date",
             "relation" :"ChargedwithCrime",
             "tail":"Bribery",
             "tail_type":"Crime"},
            {"head":"March 2022",
             "head_type":"Date",
             "relation" :"Convicted",
             "tail":"Bribery",
             "tail_type":"Crime"}
            
    ,{       "head":"March 2022",
             "head_type":"Date",
             "relation" :"Judgement",
             "tail":"sentenced to 58 years imprisonment and \
        fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.",
             "tail_type":"Decision"
        
    }]},
    
    #year
        {"input":"nov 2021 - expelled from the cpc and dismissed from the\
                    official position for bribery (cny68,400) (2017-2020). case transferred for prosecution.",
         "output": [
           { "head": "2017-2020",
            "head_type": "Date",
            "relation": "CrimeCommitted",
            "tail": "bribery",
            "tail_type": "Crime"},
             {
            "head": "November 2021",
            "head_type": "Date",
            "relation": "Judgement",
            "tail": "dismissed from the official position for bribery (cny68,400)",
            "tail_type": "Decision"
        },
        {
            "head": "November 2021",
            "head_type": "Date",
            "relation": "ChargedWithCrime",
            "tail": "bribery",
            "tail_type": "Crime"
        }

    ]},#year and sentence
     

         

    # year
     {
     "input":' may 2016 - fined rub30,000 by murom city court for organisation of illegal migration (nov 2015 - dec 2015).',
      "output":[{"head":"May 2016","head_type":"Date","relation":"Judgement","tail":"fined rub30,000","tial_type":"Decision"},{"head":"November 2015 - December 2015","head_type":"Date","relation":"CrimeCommitted","tail":"illegal migration","tial_type":"Crime"}]
    
     } 
# year

  # year and sentence 
  
# sentence

        
    ]



In [18]:
system_prompt ='''You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
Your task is to identify the entities and relations from the input text. You must generate the output as a list containing
JSON objects. Each JSON object should be in format {"head":entity,"head_type":type of entity assigned to head, "tail":entity, "tail_type":type of entity assigned to tail,"relation":relation between entities asigned to head and tail}.
'''

In [19]:
class ExtractedInfo(BaseModel):
    head: str = Field(description="extracted first or head entity like Decmeber-2022 etc.")
    head_type: str = Field(description="type of the extracted head entity like Date etc.")
    relation: str = Field(description="relation between the head and the tail entities or if tail is none then relation be also none.")
    tail: str = Field(description="extracted second or tail entity like Bribery,Fined,Imprisioned, None etc.")
    tail_type: str = Field(description="type of the extracted tail entity like Crime, Sentence, None etc.")
    
parser = JsonOutputParser(pydantic_object=ExtractedInfo)

In [20]:
# all_mrgd_ot_gpu
llm=ChatOllama(model="llama3",temperature=0,format="json")
human_prompt=PromptTemplate(template="""
Below are a list of examples,each having a JSON object with input and output as keys.Input key takes {input}
and output key generates is list of JSON having extracted entity and relations.Generate output as per this format.

examples: {examples}
Use the following relation types, don't use other relation that is not defined below:
# RELATION TYPES:
{rel_types}

For the following input, extract entities and relationships as described in examples.
{format_instructions}
Input text: {input}
In the {input}, identify all date event relationships.

"""
,input_variables=["input"],partial_variables={"examples":examples_6i,"rel_types":["Convicted","CrimeCommited","Judgement","ChargedWithCrime"],
        "format_instructions": parser.get_format_instructions()})


human_message_prompt=HumanMessagePromptTemplate(prompt=human_prompt)


# "Date" and "Crime"  with relation as "Convicted".Don't provide any other entity and relationship

In [7]:
system_message=SystemMessage(content=system_prompt)
chat_prompt=ChatPromptTemplate.from_messages([system_message,human_message_prompt])

In [21]:


# prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
# chain = chat_prompt | llm

In [22]:
df["V_BIOGRAPHY_repo"].loc[23]

' 2010 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. aug 2022 - no further information reported.'

In [23]:
 print(
chain.run(df["V_BIOGRAPHY_repo"].loc[23]))

{
    "input": {
        "2010 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. aug 2022 - no further information reported."
   	: [
            {"head": "2010", "head_type": "Date", "relation": "ChargedWithCrime", "tail": "alleged involvement in kidnapping", "tail_type": "Crime"},
            {"head": "Aug 2022", "head_type": "Date", "relation": "NoFurtherInformation", "tail": "no further information reported", "tail_type": "Report"}
        ]
    }
}


In [17]:
texts=df.loc[0:150,"V_BIOGRAPHY_repo"].to_list()

In [24]:

llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
l6=[]
bio6=[]
ind6=[]
index6=0
time6=[]
for i in texts:
    print(index6)
    
    try:
       start6=time.time()
       output=chain.run(i)
       end6=time.time()
       time6.append(end6-start6)
       l6.append(output)
       bio6.append(i)
       ind6.append(index6)
       index6=index6+1
    except:
        time6.append(0)
        index6=index6+1
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150


In [33]:
#LOAD DATASET
# sum_df = pd.read_csv(r'C:\Users\MBBLABS\Desktop\LLM_NER\LLM_Output_PT_20240626.csv', encoding_errors='replace')['Sum_Prompt_Templates']
# runset20jun = pd.read_csv(r'C:\Users\admin\Downloads\ot_anjali\all_mrgd_ot_gpu.csv')#['sent']

In [32]:
# sum_df = open('LLM_Output_PT_20240626.csv', encoding="cp437", errors='ignore')

In [31]:
df_res=pd.DataFrame({})
df_res["Biography"]=bio6
df_res["groups"]=l6
df_res["index"]=ind6


In [32]:
df_res["time"]=time6

In [33]:
df_res

,Biography,groups,index,time
0,mar 2022 - no further information reported. [...,"{\n ""output"": [\n {\n ""he...",0,6.380915
1,dec 2019 - pleaded not guilty to the criminal...,"{""head"": ""July 2019-October 2019"", ""head_type""...",1,5.222504
2,dec 2018 - reportedly detained. sentenced to ...,"{""head"": ""Dec 2018"", ""head_type"": ""Date"", ""rel...",2,5.347117
3,jun 2018 - declared as proclaimed offender by...,"{\n ""input"": [\n {\n ""hea...",3,8.150769
4,aug 2013 - declared as proclaimed offender by...,"{\n ""input"": [\n {\n ""hea...",4,8.528394
...,...,...,...,...
146,nov 2017 - detained by pakistan rangers sindh...,"{""head"": ""Nov 2017"", ""head_type"": ""Date"", ""rel...",146,5.320444
147,apr 2014 - sentenced by baiyin municipal inte...,"{\n ""output"": [\n {\n ""he...",147,9.625437
148,may 2009 - reportedly criminally charged with...,"{""head"": ""May 2009"", ""head_type"": ""Date"", ""rel...",148,5.482109
149,may 2022 - convicted of income tax and goods ...,"{ ""head"": ""May 2022"", ""head_type"": ""Date"", ""re...",149,5.318701


In [34]:
df_res.to_csv("final_res_ex_6i.csv")

## Date of crime prompt

In [93]:
# doc=[      {"input":" feb 2023 - reportedly fined bdt0.1m and sentenced to 10 years of \
#     imprisonment for trafficking narcotics and seized (0.5m yaba tablets).",
#     "output": [{"head":None,"head_type":None,"relation":None,"tail":None,"tail_type":None}]},
 
#  {"input":' 2010 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. aug 2022 - no further information reported.',
#             "output": [{"head":None,"head_type":None,"relation":None,"tail":None,"tail_type":None}]},
#  {"input":" jun 2022 - pleaded guilty at the magistrate court to the criminal charges of instigating employees in practicing dentistry without \
#           authorisation from the brunei medical board (2019 - 2020). fined bnd24,000, in default, 24 months imprisonment.",
#   "output": [{
#             "head": "2019 - 2020",
#             "head_type": "Date",
#             "relation": "CrimeCommited",
#             "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board ",
#             "tail_type": "Crime"
#         }]},
#      {"input":" may 2012 - sentenced by xihu district court to 3 years imprisonment for extortion (cny6.57m) (sep 2011).",
#                 "output":[{"head":"September 2011","head_type":"Date","relation":"CrimeCommitted","tail":"extortion","tail_type":"Crime"}]},


#   {  "input": 'Oct 2022 - sentenced by Sampit District Court to 5 years \
#             imprisonment and fined IDR1b in lieu of additional 3 months imprisonment for methamphetamine trafficking (Feb - 2021). \
#             and last year he got an award as the Best Talent',
#   "output":[{"head":"February 2021",
#          "head_type":"Date",
#          "relation":"CrimeCommited",
#          "tail":"methamphetamine trafficking",
#          "tail_type":"Crime"}]},{"input": ' 2009 - reportedly declared as proclaimed offender by ghotki court for alleged involvement in dacoity. jun 2022 - no further information reported.'
# ,"output": [{"head":None,"head_type":None,"relation":None,"tail":None,"tail_type":None}]}]

 


 
 
 

In [211]:

doc_2=[{"input":" apr 2022 - pleaded guilty at the sessions court to the criminal charges \
of bribery (myr20,000) (jun 2021). fined myr5,000 in default 1 month imprisonment.",
        "output":[{"head":"June 2021","head_type":"Date","relation":"CrimeCommitted","tail":"bribery","tail_type":"Crime"}]}

       
    ,      {"input":" feb 2023 - reportedly fined bdt0.1m and sentenced to 10 years of \
    imprisonment for trafficking narcotics and seized (0.5m yaba tablets).",
    "output": [{"head":None,"head_type":None,"relation":None,"tail":None,"tail_type":None}]},
 
 {"input":' 2010 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. aug 2022 - no further information reported.',
            "output": [{"head":None,"head_type":None,"relation":None,"tail":None,"tail_type":None}]},
 {"input":" jun 2022 - pleaded guilty at the magistrate court to the criminal charges of instigating employees in practicing dentistry without \
          authorisation from the brunei medical board (2019 - 2020). fined bnd24,000, in default, 24 months imprisonment.",
  "output": [{
            "head": "2019 - 2020",
            "head_type": "Date",
            "relation": "CrimeCommited",
            "tail": "instigating employees in practicing dentistry without authorisation from the brunei medical board ",
            "tail_type": "Crime"
        }]},
     {"input":" may 2012 - sentenced by xihu district court to 3 years imprisonment for extortion (cny6.57m) (sep 2011).",
                "output":[{"head":"September 2011","head_type":"Date","relation":"CrimeCommitted","tail":"extortion","tail_type":"Crime"}]},


  {  "input": 'Oct 2022 - sentenced by Sampit District Court to 5 years \
            imprisonment and fined IDR1b in lieu of additional 3 months imprisonment for methamphetamine trafficking (Feb - 2021). \
            and last year he got an award as the Best Talent',
  "output":[{"head":"February 2021",
         "head_type":"Date",
         "relation":"CrimeCommited",
         "tail":"methamphetamine trafficking",
         "tail_type":"Crime"}]},{"input": ' 2009 - reportedly declared as proclaimed offender by ghotki court for alleged involvement in dacoity. jun 2022 - no further information reported.'
,"output": [{"head":None,"head_type":None,"relation":None,"tail":None,"tail_type":None}]}]

 


 
 

In [212]:
system_prompt ='''You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
Your task is to identify the entities and relations from the input text. You must generate the output as a list containing
JSON objects. Each JSON object should be in format {"head":entity,"head_type":type of entity assigned to head, "tail":entity, "tail_type":type of entity assigned to tail,"relation":relation between entities asigned to head and tail}.
'''

In [213]:
class ExtractedInfo(BaseModel):
    head: str = Field(description="extracted first or head entity like Decmeber-2022 etc.")
    head_type: str = Field(description="type of the extracted head entity like Date etc.")
    relation: str = Field(description="relation between the head and the tail entities or if tail is none then relation be also none.")
    tail: str = Field(description="extracted second or tail entity like Bribery,Fined,Imprisioned, None etc.")
    tail_type: str = Field(description="type of the extracted tail entity like Crime, Sentence, None etc.")
    
parser = JsonOutputParser(pydantic_object=ExtractedInfo)

In [204]:
# # all_mrgd_ot_gpu
# llm=ChatOllama(model="llama3",temperature=0,format="json")
# human_prompt=PromptTemplate(template="""
# Below are a list of examples,each having a JSON object with input and output as keys.Input key takes {input}
# and output key generates is list of JSON having extracted entity and relations.Generate output as per this format.

# examples: {examples}
# Use the following relation types, don't use other relation that is not defined below:
# # RELATION TYPES:
# {rel_types}

# For the following input, extract entities and relationships as described in examples.
# {format_instructions}
# Input text: {input}
# Extract the only dates inside "()" from the {input} and create relations using them only.If there no dates inside "()" give output as dictionay with keys as
# ["head","head_type","relation","tail","tail_type"] and their values as [None,None,None,None,None].

# Special Note:DO NOT TAKE ANY OTHER DATE EXCEPT THE DATE INSIDE PARANTHESIS.
#              DATE SHOULD BE IN NUMBERS, NOT ALPHABETS.IF ALPHABETS THEN DO NOT TAKE IT 
# """
# ,input_variables=["input"],partial_variables={"examples":doc,"rel_types":["CrimeCommited"],
#         "format_instructions": parser.get_format_instructions()})


# human_message_prompt=HumanMessagePromptTemplate(prompt=human_prompt)


# # "Date" and "Crime"  with relation as "Convicted".Don't provide any other entity and relationship

In [214]:
# all_mrgd_ot_gpu
llm=ChatOllama(model="llama3",temperature=0,format="json")
human_prompt=PromptTemplate(template="""
Below are a list of examples,each having a JSON object with input and output as keys.Input key takes {input}
and output key generates is list of JSON having extracted entity and relations.Generate output as per this format.

examples: {examples}
Use the following relation types, don't use other relation that is not defined below:
# RELATION TYPES:
{rel_types}

For the following input, extract entities and relationships as described in examples.
{format_instructions}
Input text: {input}
Extract meaningful string inside "()" if it is date and create relations using it only.If there no dates inside "()" give output as dictionay with keys as
["head","head_type","relation","tail","tail_type"] and their values as [None,None,None,None,None].

Special Note:DO NOT TAKE ANY OTHER DATE EXCEPT THE DATE INSIDE PARANTHESIS.
              
"""
,input_variables=["input"],partial_variables={"examples":doc_2,"rel_types":["CrimeCommited"],
        "format_instructions": parser.get_format_instructions()})


human_message_prompt=HumanMessagePromptTemplate(prompt=human_prompt)


# "Date" and "Crime"  with relation as "Convicted".Don't provide any other entity and relationship

In [215]:
system_message=SystemMessage(content=system_prompt)
chat_prompt=ChatPromptTemplate.from_messages([system_message,human_message_prompt])

In [216]:
chain = LLMChain(llm=llm, prompt=chat_prompt)

In [217]:
print(df["V_BIOGRAPHY_repo"].loc[10])

 2014 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. jun 2022 - no further information reported.


In [209]:
# doc_2 , 2nd prompt

print("2014 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. jun 2022 - no further information reported.")
print(
chain.run(df["V_BIOGRAPHY_repo"].loc[10]
         ))

print(' 2009 - reportedly declared as proclaimed offender by ghotki court for alleged involvement in dacoity. jun 2022 - no further information reported.')
print(
chain.run( ' 2009 - reportedly declared as proclaimed offender by ghotki court for alleged involvement in dacoity. jun 2022 - no further information reported.'
))
print("feb 2021 - criminally charged by the sessions court with bribery (myr3,000) (dec 2016). pleaded not guilty.\
bail allowed. case re-mention scheduled for mar 2021.")
print(
chain.run("feb 2021 - criminally charged by the sessions court with bribery (myr3,000) (dec 2016). pleaded not guilty.\
bail allowed. case re-mention scheduled for mar 2021."))


print("oct 2022 - criminally charged at the\
magistrate court with cannabis trafficking (5,152g) (sep 2022). no plea recorded. case mention scheduled for dec 2022.")

print(
chain.run("oct 2022 - criminally charged at the\
magistrate court with cannabis trafficking (5,152g) (sep 2022). no plea recorded. case mention scheduled for dec 2022."))



print(chain.run(" apr 2022 - pleaded guilty at the sessions court to the criminal charges of bribery (myr20,000)\
(jun 2021). fined myr5,000 in default 1 month imprisonment."))


print("jun 2011 - reportedly under investigation for graft during the distribution of subsidised cooking oil (idr1.3b) (2008). dec 2012 - sentenced to 1 year imprisonment and fined idr50m. jan 2019 - no further information reported.")
      
print(
chain.run("jun 2011 - reportedly under investigation for graft during the distribution of subsidised cooking oil (idr1.3b) (2008). dec 2012 - sentenced to 1 year imprisonment and fined idr50m. jan 2019 - no further information reported."))


print(" jun 2022 - civil lawsuit filed by the bank pembangunan malaysia (bpmb) for allegedly conspiring and using unlawful means to defraud the development financial institution (dfi) and conceal such fraud and its proceeds from bpmb (myr400m) and bribery (myr8m) (jul 2012). also, reportedly, ex-parte mareva injunction filed by the bpmb. jul 2022 - reportedly an ex-parte mareva injunction obtained from the high court by \
the bpmb to freeze the assets. apr 2023 - reportedly reprimanded and fined myr25,000 by mybursa for breaching market listing requirements (may 2021).")
print(
chain.run(" jun 2022 - civil lawsuit filed by the bank pembangunan malaysia (bpmb) for allegedly conspiring and using unlawful means to defraud the development financial institution (dfi) and conceal such fraud and its proceeds from bpmb (myr400m) and bribery (myr8m) (jul 2012). also, reportedly, ex-parte mareva injunction filed by the bpmb. jul 2022 - reportedly an ex-parte mareva injunction obtained from the high court by \
the bpmb to freeze the assets. apr 2023 - reportedly reprimanded and fined myr25,000 by mybursa for breaching market listing requirements (may 2021)."))


print(" apr 2022 - pleaded guilty at the sessions court to the criminal charges \
of bribery (myr20,000) (jun 2021). fined myr5,000 in default 1 month imprisonment.")    
print(chain.run(" apr 2022 - pleaded guilty at the sessions court to the criminal charges \
of bribery (myr20,000) (jun 2021). fined myr5,000 in default 1 month imprisonment."))


print( "accused of participating in a scheme to bribe officials at us citizenship and immigration services to illegally obtain green cards, and officials at the maryland comptroller's office to receive abatement of state taxes (apr 2005 - 2007). sep 2007 - arrested. charged with conspiracy to bribe public officials. sep 2008 - sentenced to 3 years in prison and 3 years supervised release. \
 agreed to forfeit usd323,565. mar 2019 - no further information reported.")
print(
chain.run( "accused of participating in a scheme to bribe officials at us citizenship and immigration services to illegally obtain green cards, and officials at the maryland comptroller's office to receive abatement of state taxes (apr 2005 - 2007). sep 2007 - arrested. charged with conspiracy to bribe public officials. sep 2008 - sentenced to 3 years in prison and 3 years supervised release. \
 agreed to forfeit usd323,565. mar 2019 - no further information reported."))


2014 - reportedly declared as proclaimed offender by khairpur court for alleged involvement in kidnapping. jun 2022 - no further information reported.
{
    "output": [
        {
            "head": "June 2022",
            "head_type": "Date",
            "relation": "CrimeCommitted",
            "tail": "kidnapping",
            "tail_type": "Crime"
        }
    ]
}
 2009 - reportedly declared as proclaimed offender by ghotki court for alleged involvement in dacoity. jun 2022 - no further information reported.
{
    "output": [
        {
            "head": "June 2022",
            "head_type": "Date",
            "relation": "CrimeCommitted",
            "tail": "dacoity",
            "tail_type": "Crime"
        }
    ]
}
feb 2021 - criminally charged by the sessions court with bribery (myr3,000) (dec 2016). pleaded not guilty.bail allowed. case re-mention scheduled for mar 2021.
{
    "output": [
        {
            "head": "December 2016",
            "head_type": "Date",
    

In [218]:
df["V_BIOGRAPHY_repo"].loc[9]

' 2009 - reportedly declared as proclaimed offender by ghotki court for alleged involvement in dacoity. jun 2022 - no further information reported.'

In [219]:
texts=df.loc[0:150,"V_BIOGRAPHY_repo"].to_list()

In [220]:

llm = ChatOllama(model = "llama3", temperature=0,format="json")
chain = LLMChain(llm=llm, prompt=chat_prompt)
l_d=[]
bio_d=[]
ind_d=[]
index_d=0
time_d=[]
for i in texts:
    print(index_d)
    
    try:
       start_d=time.time()
       output=chain.run(i)
       end_d=time.time()
       time_d.append(end_d-start_d)
       l_d.append(output)
       bio_d.append(i)
       ind_d.append(index_d)
       index_d=index_d+1
    except:
        time_d.append(0)
        index_d=index_d+1
        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150


In [221]:
df_res=pd.DataFrame({})
df_res["Biography"]=bio_d
df_res["groups"]=l_d
df_res["index"]=ind_d


In [222]:
df_res["time"]=time_d

In [223]:
df_res.to_csv("date_of_crime.csv")

In [39]:
# f_df = pd.DataFrame()
# # for i in range(len(sum_df)):
# for i in range(2, 5):
#     print(i)
#     st_t = time.time()
#     recusi = recurs_split(sum_df[i])
#     response = chain.invoke(entity_types = entity_types, relation_types = relation_types, examples = examples, text = recusi)
#     et_t = time.time()
#     total_time = (et_t-st_t)/60
#     print('time taken for model', total_time)
#     a = pd.DataFrame([[sum_df[i], response, total_time]], columns = ['sent', 'output_model', 'time taken'])
#     f_df = pd.concat([f_df, a])
#     if i%10==0:
#          nm  = 'ot_ner_ks/llm_ed_'+str(i+1)+'_08july24.csv'
#          f_df.to_csv(nm)
#     print('for', i, 'concat done')
# f_df.to_csv( 'ot_ner_ks/llm_ed_FINAL_08july24.csv')

## The end

In [47]:
a['output_model'].tolist()

["Here is the extracted entities and relations:\n\n[{'head': 'error message', 'head type': 'text', 'relation': 'none', 'tail': 'none', 'tail type': 'none'}, \n {'head': 'news article information', 'head type': 'information', 'relation': 'lack of', 'tail': 'provided text', 'tail type': 'text'}, \n {'head': 'summary', 'head type': 'output', 'relation': 'unable to create', 'tail': 'as instructed', 'tail type': 'instruction'}, \n {'head': 'valid news article', 'head type': 'news article', 'relation': 'requirement for', 'tail': 'I would be happy to assist you in summarizing it according to the guidelines', 'tail type': 'guideline'}]"]

In [77]:
f_df.to_csv('ot_anjali/otfinal.csv')

In [85]:
f_df[f_df['sent'].str.contains('NANDAR')==True].to_csv('ot_anjali/otfinal2.csv')#['output_model'].tolist()

In [45]:
f_df.to_csv('ot_anjali/lastrun.csv')

In [33]:
f_df.to_csv('ot_anjali/llm_ed_FINAL_25jun_run3_905.csv')

In [37]:
bb = f_df.tail(1)['sent'].tolist()[0]

In [20]:
pd.read_csv('ot_anjali/llm_ed_FINAL_25JUN24_r2.csv').tail(1)['sent'].tolist()

['[BIOGRAPHY] To be determined. [IDENTIFICATION] San Nwe (father). Reported age: 30 (reported 2022). [REPORTS] Sep 2022 - escaped prison and wanted by Myanmar Police Force.']

In [38]:
runset201.index(bb)

3871

In [40]:
f_df.to_csv('llm_ed_FINAL_25JUN24_2878.csv')

In [41]:
f_df.shape

(2878, 3)

In [14]:
f_df.to_csv('llm_ed_FINAL_20jun24_f22.csv')

In [18]:
f_df.to_csv('llm_ed_24jun_f.csv')

In [2]:
import pandas as pd
import os

In [4]:
os.listdir('ot_anjali')

['lastrun.csv',
 'llm_ed_1001_24jun24_f.csv',
 'llm_ed_101_24jun24_f.csv',
 'llm_ed_101_25jun24_f.csv',
 'llm_ed_101_25jun24_f2.csv',
 'llm_ed_1051_24jun24_f.csv',
 'llm_ed_1101_24jun24_f.csv',
 'llm_ed_1151_24jun24_f.csv',
 'llm_ed_1201_24jun24_f.csv',
 'llm_ed_1251_24jun24_f.csv',
 'llm_ed_1301_24jun24_f.csv',
 'llm_ed_1351_24jun24_f.csv',
 'llm_ed_1401_24jun24_f.csv',
 'llm_ed_1451_24jun24_f.csv',
 'llm_ed_1501_24jun24_f.csv',
 'llm_ed_151_24jun24_f.csv',
 'llm_ed_151_25jun24_f.csv',
 'llm_ed_151_25jun24_f2.csv',
 'llm_ed_1551_24jun24_f.csv',
 'llm_ed_1601_24jun24_f.csv',
 'llm_ed_1651_24jun24_f.csv',
 'llm_ed_1701_24jun24_f.csv',
 'llm_ed_1751_24jun24_f.csv',
 'llm_ed_1801_24jun24_f.csv',
 'llm_ed_1851_24jun24_f.csv',
 'llm_ed_1901_24jun24_f.csv',
 'llm_ed_1951_24jun24_f.csv',
 'llm_ed_1_24jun24_f.csv',
 'llm_ed_1_25jun24_f.csv',
 'llm_ed_1_25jun24_f2.csv',
 'llm_ed_2001_24jun24_f.csv',
 'llm_ed_201_24jun24_f.csv',
 'llm_ed_201_25jun24_f.csv',
 'llm_ed_201_25jun24_f2.csv',
 'llm_ed

In [ ]:
#last run

In [18]:
a = pd.read_csv('ot_anjali/lastrun.csv')

In [19]:
b = pd.read_csv('ot_anjali/llm_ed_FINAL_20jun24_f33a.csv')

In [9]:
#before last run

In [20]:

c = pd.read_csv('ot_anjali/llm_ed_501_25jun24_f2.csv')

In [21]:

d =pd.read_csv('ot_anjali/llm_ed_FINAL_25jun_run3_905.csv')

In [22]:

e = pd.read_csv('ot_anjali/llm_ed_401_25jun24_f.csv')

In [23]:

f = pd.read_csv('ot_anjali/llm_ed_FINAL_25JUN24_r2.csv')

In [24]:

g =pd.read_csv('ot_anjali/llm_ed_201_24jun24_f.csv')

In [25]:

h = pd.read_csv('ot_anjali/llm_ed_FINAL_25JUN24_2878.csv')

In [26]:

ij = pd.read_csv('ot_anjali/llm_ed_FINAL_20jun24_f22.csv')

In [28]:

kl = pd.read_csv('ot_anjali/llm_ed_24jun_f.csv')

In [33]:
alcnct = pd.concat([a,b,c,d,e,f,g,h,ij,kl])

In [36]:
alcnct[alcnct['sent'].duplicated()==False].shape

(4417, 4)

In [42]:
alcnct[alcnct[['sent','output_model']].duplicated()==False].to_csv('all_mrgd_ot_gpu.csv')

In [47]:
pd.read_csv('ot_anjali/all_mrgd_ot_gpu.csv')['time taken'].describe()

count    4417.000000
mean        0.420733
std         3.959685
min         0.072467
25%         0.181958
50%         0.256971
75%         0.368348
max       195.634282
Name: time taken, dtype: float64